<h1>Импорты</h1>

In [196]:
import pandas as pd
import numpy as np
import pyodbc
import random
from sklearn.preprocessing import OrdinalEncoder
from datetime import datetime

In [197]:
# Connect to old database
db_conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=BAMBI;'
                      'Database=Pro_t;'
                      'Trusted_Connection=yes;')

In [627]:
# Connect to new database
try:
    conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=BAMBI;'
                      'Database=DocumentsKM;'
                      'Trusted_Connection=yes;')
    cursor = conn.cursor()
except Exception as err:
    cursor = None
    print("Connection error:", err)
    
# Check if the connection was valid
if cursor != None:
    print("Connection successful")

Connection successful


In [614]:
def to_pascal_case(snake_str):
    components = snake_str.split('_')
    return components[0].title() + ''.join(x.title() for x in components[1:])

In [621]:
def execute_values(df, table_snake_case):
    df.columns = map(to_pascal_case, df.columns)
    df = df.replace({False: 0, True: 1})
    table = to_pascal_case(table_snake_case)
    cursor.execute(f"SET IDENTITY_INSERT dbo.{table} ON")
    tuples = [tuple(x) for x in df.to_numpy()]
    cols = ','.join(list(df.columns))
    for entry in tuples:
#         query  = "INSERT INTO dbo.%s(%s) VALUES %s" % (table, cols, entry)
#         query  = "INSERT INTO dbo.%s(%s) VALUES ( " % (table, cols)
        query  = "insert into dbo.%s(%s) values (" % (table, cols)
        num = 1
        for e in entry:
            if num == len(entry):
                query += "?"
            else:
                query += "?, "
            num += 1
        query += ")"
        print(query)
        print(*entry)
        cursor.execute(query, *entry)
    cursor.execute(f"SET IDENTITY_INSERT dbo.{table} OFF")
    conn.commit()

<h1>Должности</h1>

In [201]:
df = pd.read_json('post.json')

In [202]:
df["name"] = df["name"].str.strip()

In [203]:
cursor.execute(open("sql_server/25.sql", "r").read())
conn.commit()

In [204]:
execute_values(df, "positions")

<h1>Отделы</h1>

In [205]:
df = pd.read_json('department.json')

In [206]:
df = df[df["enable"]]

In [207]:
df["name"] = df["name"].str.strip()
df["reduction"] = df["reduction"].str.strip()

In [208]:
df.loc[62, "name"] = "Строительный отдел №1-а"
df.loc[64, "name"] = "Строительный отдел №1-б"

In [209]:
df = df.drop(["enable"], axis=1)

In [210]:
df = df.rename(columns={"id" : "code",
                        "reduction": "short_name"
                       })

In [211]:
df.insert(0, 'id', range(1, len(df) + 1))

In [212]:
df.loc[:, ["id", "code"]].to_csv("departments.csv")

In [213]:
cursor.execute(open("sql_server/30.sql", "r").read())
conn.commit()

In [214]:
execute_values(df, "departments")

<h1>Исполнители</h1>

In [215]:
df = pd.read_json('staff.json')

In [216]:
df_dep = df['department'].apply(pd.Series)

In [217]:
df_dep = df_dep.drop(["name", "reduction", "enable"], axis=1)

In [218]:
df_dep = df_dep.rename(columns={"id": "code"})

In [219]:
dep = pd.read_csv("departments.csv")
dep = dep.drop(["Unnamed: 0"], axis=1)

In [220]:
for index, row in df_dep.iterrows():
    try:
        df_dep.loc[index, "department_id"] = dep.loc[dep["code"] == row["code"], "id"].values[0]
    except:
        df_dep.loc[index, "department_id"] = None

In [221]:
df_dep = df_dep.drop(["code"], axis=1)

In [222]:
df["department_id"] = df_dep

In [223]:
df_post = df['post'].apply(pd.Series)

In [224]:
df_post = df_post.drop(["name"], axis=1)

In [225]:
df_post = df_post.rename(columns={"id": "position_id"})

In [226]:
df["position_id"] = df_post

In [227]:
df = df.drop(["tableNumber", "userName", "createdDate", "dismissedDate", "vacation", "post", "department"], axis=1)

In [228]:
df = df.dropna()

In [229]:
df["name"] = df["fullName"].str.partition().loc[:, 0]

In [230]:
df.loc[3078, "name"] = "ОАО МАГНИТОГОРСКИЙ ГИПРОМЕЗ"

In [231]:
df["id"].to_csv("employee_ids.csv")

In [232]:
cursor.execute(open("sql_server/26.sql", "r").read())
conn.commit()

In [233]:
execute_values(df, "employees")

<h1>Пользователи</h1>

In [234]:
cursor.execute(open("sql_server/xx_users.sql", "r").read())
conn.commit()

<h1>Значения по умолчанию</h1>

In [270]:
cursor.execute(open("sql_server/xx_default.sql", "r").read())
conn.commit()

<h1>Условия эксплуатации</h1>

In [236]:
df1 = pd.DataFrame({'id': [1, 2, 3, 4], 'name': ["неагрессивная", "слабоагрессивная", "среднеагрессивная", "сильноагрессивная"]})

In [237]:
df2 = pd.DataFrame({'id': [1, 2, 3], 'name': ["внутри помещения", "снаружи помещения", "в жидкостях"]})

In [238]:
df3 = pd.DataFrame({'id': [1, 2, 3, 4, 5], 'name': ["нет газов (вода)", "газы группы А (кислоты)", "газы группы B, C и D (щелочи)", "бензин", "масло"]})

In [239]:
df4 = pd.DataFrame({'id': [1, 2, 3, 4], 'name': ["сталь без покрытия", "оцинкованная сталь", "сталь горячего цинкования", "сталь с газотермическим напылением цинка"]})

In [240]:
df5 = pd.DataFrame({'id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17], 'name': ["ПФ", "АУ", "ЭЭ", "МА", "МБ", "НЦ", "ФФ", "ПА", "ПЭ", "ПБ", "ПВ", "СВ", "ПУ", "ЭП", "ПС", "КО", "АЦ"]})

In [241]:
df6 = pd.DataFrame({'id': [1, 2, 3, 4], 'name': ["отсутствуют", "сдвигоустойчивые соединения", "фланцевые соединения", "сдвигоустойчивые и фланцевые соединения"]})

In [242]:
df5.loc[:, ["id", "name"]].to_csv("paintwork.csv")

In [243]:
cursor.execute(open("sql_server/y_1.sql", "r").read())
cursor.execute(open("sql_server/y_2.sql", "r").read())
cursor.execute(open("sql_server/y_3.sql", "r").read())
cursor.execute(open("sql_server/y_4.sql", "r").read())
cursor.execute(open("sql_server/y_5.sql", "r").read())
cursor.execute(open("sql_server/y_6.sql", "r").read())
conn.commit()

In [244]:
execute_values(df1, "env_aggressiveness")
execute_values(df2, "operating_areas")
execute_values(df3, "gas_groups")
execute_values(df4, "construction_materials")
execute_values(df5, "paintwork_types")
execute_values(df6, "high_tensile_bolts_types")

<h1>Стойкость лакокрасочных покрытий</h1>

In [245]:
df = pd.DataFrame({'id': [1, 2, 3, 4, 5, 6, 7, 8, 9], 'name': ["П", "А", "Х", "В", "Б", "М", "ХЩ", "ХК", "Т"]})

In [246]:
df.loc[:, ["id", "name"]].to_csv("paintwork_fastness.csv")

In [247]:
cursor.execute(open("sql_server/y_7.sql", "r").read())
conn.commit()

In [248]:
execute_values(df, "paintwork_fastness")

<h1>Контроль сварки</h1>

In [628]:
df = pd.DataFrame({'Id': [1, 2, 3, 4], 'Name': ["не требуется", "опрыскиванием керосином", "ультразвуком", "гамма-излучением"]})

In [629]:
cursor.execute(open("sql_server/yy_1.sql", "r").read())
conn.commit()

ProgrammingError: ('42S01', '[42S01] [Microsoft][ODBC SQL Server Driver][SQL Server]В базе данных уже существует объект с именем "WeldingControl". (2714) (SQLExecDirectW)')

In [630]:
execute_values(df, "WeldingControl")

insert into dbo.Weldingcontrol(Id,Name) values (?, ?)
1 не требуется
insert into dbo.Weldingcontrol(Id,Name) values (?, ?)
2 опрыскиванием керосином
insert into dbo.Weldingcontrol(Id,Name) values (?, ?)
3 ультразвуком
insert into dbo.Weldingcontrol(Id,Name) values (?, ?)
4 гамма-излучением


<h1>Типы документов</h1>

In [252]:
df = pd.DataFrame({'id': [1, 2, 3, 4, 5], 'code': ["л.", "СМ", "ВМП", "СМС", "РР"], 'name': ["Листы основного комплекта", "Спецификация металлопроката", "Ведомость металлоконструкций по видам профилей", "Сводная спецификация металлопроката", "Расчет металлоконструкций"]})

In [253]:
cursor.execute(open("sql_server/x.sql", "r").read())
conn.commit()

In [254]:
execute_values(df, "doc_types")

<h1>Грунтовка</h1>

In [261]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Грунт_аз', db_conn)

In [262]:
df = df.replace("", np.nan)

In [263]:
df["Приоритет"] = df["Приоритет"].map(lambda x: 0 if pd.isna(x) else x)

In [264]:
df = df.rename(columns={"поз": "id",
                        "Группа_гр": "group_num",
                        "Наим_гр": "name",
                        "Можно_гр": "can_be_primed",
                        "Приоритет": "priority"
                       })

In [265]:
cursor.execute(open("sql_server/4.sql", "r").read())
conn.commit()

In [269]:
execute_values(df, "primer")

<h1>Степени очистки антикоррозионной защиты</h1>

In [271]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Степени_оч_аз', db_conn)

In [272]:
df["Наим_оч"] = df["Наим_оч"].str.strip()

In [273]:
df = df.rename(columns={"Степень_оч": "id",
                        "Наим_оч": "name"
                       })

In [274]:
cursor.execute(open("sql_server/44.sql", "r").read())
conn.commit()

In [275]:
execute_values(df, "corr_prot_cleaning_degrees")

<h1>Варианты антикоррозионной защиты </h1>

In [597]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Варианты_аз', db_conn)

In [598]:
df["Тип_лп"] = df["Тип_лп"].str.strip()
df["Стойкость_лп"] = df["Стойкость_лп"].str.strip()

In [599]:
df = df.replace("", np.nan)

In [600]:
df = df.drop_duplicates()

In [601]:
df = df.drop_duplicates(subset=["Зона_экс", "Группа_газов", "Агресс_ср", "Матер_к", "Тип_лп", "Группа_лп", "Стойкость_лп"])

In [602]:
paintwork = pd.read_csv("paintwork.csv")
paintwork = paintwork.drop(["Unnamed: 0"], axis=1)

In [603]:
for index, row in df.iterrows():
    try:
        df.loc[index, "paintwork_type_id"] = paintwork.loc[paintwork["name"] == row["Тип_лп"], "id"].values[0]
    except:
        df.loc[index, "paintwork_type_id"] = None

In [604]:
paintwork_fastness = pd.read_csv("paintwork_fastness.csv")
paintwork_fastness = paintwork_fastness.drop(["Unnamed: 0"], axis=1)

In [605]:
for index, row in df.iterrows():
    try:
        df.loc[index, "paintwork_fastness_id"] = paintwork_fastness.loc[paintwork_fastness["name"] == row["Стойкость_лп"], "id"].values[0]
    except:
        df.loc[index, "paintwork_fastness_id"] = None

In [606]:
df = df.drop(["Тип_лп", "Стойкость_лп"], axis=1)

In [607]:
df["Группа_газов"] += 1
df["Агресс_ср"] += 1
df["Матер_к"] += 1

In [608]:
df = df.rename(columns={"Вариант": "id",
                        "Зона_экс": "operating_area_id",
                        "Группа_газов": "gas_group_id",
                        "Агресс_ср": "env_aggressiveness_id",
                        "Матер_к": "construction_material_id",
                        "Группа_лп": "paintwork_group",
                        "Кол_слоев_лп": "paintwork_num_of_layers",
                        "Толщ_лпгр": "paintwork_primer_thickness",
                        "Кол_слоев_гр": "primer_num_of_layers",
                        "Степень_оч": "cleaning_degree_id",
                        "Статус_аз": "status"
                       })

In [609]:
df = df.where(pd.notnull(df), None)

In [610]:
cursor.execute(open("sql_server/1.sql", "r").read())
conn.commit()

In [622]:
execute_values(df, "corr_prot_variants")

insert into dbo.CorrProtVariants(Id,Operatingareaid,Gasgroupid,Envaggressivenessid,Constructionmaterialid,Paintworkgroup,Paintworknumoflayers,Paintworkprimerthickness,Primernumoflayers,Cleaningdegreeid,Status,Paintworktypeid,Paintworkfastnessid) values (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
1.0 1.0 1.0 1.0 1.0 1.0 1.0 30.0 1.0 3.0 1.0 1.0 1.0
insert into dbo.CorrProtVariants(Id,Operatingareaid,Gasgroupid,Envaggressivenessid,Constructionmaterialid,Paintworkgroup,Paintworknumoflayers,Paintworkprimerthickness,Primernumoflayers,Cleaningdegreeid,Status,Paintworktypeid,Paintworkfastnessid) values (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
2.0 1.0 1.0 1.0 1.0 1.0 1.0 30.0 1.0 3.0 1.0 2.0 1.0
insert into dbo.CorrProtVariants(Id,Operatingareaid,Gasgroupid,Envaggressivenessid,Constructionmaterialid,Paintworkgroup,Paintworknumoflayers,Paintworkprimerthickness,Primernumoflayers,Cleaningdegreeid,Status,Paintworktypeid,Paintworkfastnessid) values (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
3.0 1.0 1.0 1.0

ProgrammingError: ('42000', '[42000] [Microsoft][ODBC SQL Server Driver][SQL Server]Неверный входящий поток табличных данных (TDS) по протоколу удаленного вызова процедур (RPC). Параметр 9 (""): переданное значение не является допустимым экземпляром типа данных float. Проверьте исходные данные на наличие неверных значений. Пример недопустимого значения: числовые данные, масштаб которых больше, чем точность. (8023) (SQLExecDirectW)')

<h1>Способы антикоррозионной защиты</h1>

In [314]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Способы_аз', db_conn)

In [315]:
df["Наим_сп"] = df["Наим_сп"].str.strip()

In [316]:
df["Агресс_ср"] += 1
df["Матер_к"] += 1

In [317]:
df.insert(0, 'id', range(1, len(df) + 1))

In [318]:
df = df.rename(columns={"Агресс_ср": "env_aggressiveness_id",
                        "Матер_к": "construction_material_id",
                        "Наим_сп": "name",
                        "Статус_сп": "status"
                       })

In [319]:
cursor.execute(open("sql_server/19.sql", "r").read())
conn.commit()

In [320]:
execute_values(df, "corr_prot_methods")

ProgrammingError: ('42000', '[42000] [Microsoft][ODBC SQL Server Driver][SQL Server]IDENTITY_INSERT уже установлен в ON для таблицы "DocumentsKM.dbo.CorrProtVariants". Невозможно выполнить операцию SET для таблицы "dbo.CorrProtMethods". (8107) (SQLExecDirectW)')

<h1>Покрытия антикоррозионной защиты</h1>

In [622]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Покрытия_аз', db_conn)

In [623]:
df = df.replace("", np.nan)

In [624]:
df["Тип_лп"] = df["Тип_лп"].str.strip()
df["Стойкость_лп"] = df["Стойкость_лп"].str.strip()
df["Наим_лп"] = df["Наим_лп"].str.strip()

In [625]:
df["Приоритет"] = df["Приоритет"].map(lambda x: 0 if pd.isna(x) else x)

In [626]:
df = df.drop_duplicates(subset=["Тип_лп", "Группа_лп", "Стойкость_лп", "Наим_лп"])

In [627]:
df = df.where(pd.notnull(df), None)

In [628]:
paintwork = pd.read_csv("paintwork.csv")
paintwork = paintwork.drop(["Unnamed: 0"], axis=1)

In [629]:
for index, row in df.iterrows():
    try:
        df.loc[index, "paintwork_type_id"] = paintwork.loc[paintwork["name"] == row["Тип_лп"], "id"].values[0]
    except:
        df.loc[index, "paintwork_type_id"] = None

In [630]:
paintwork_fastness = pd.read_csv("paintwork_fastness.csv")
paintwork_fastness = paintwork_fastness.drop(["Unnamed: 0"], axis=1)

In [631]:
for index, row in df.iterrows():
    try:
        df.loc[index, "paintwork_fastness_id"] = paintwork_fastness.loc[paintwork_fastness["name"] == row["Стойкость_лп"], "id"].values[0]
    except:
        df.loc[index, "paintwork_fastness_id"] = None

In [632]:
df = df.drop(["Тип_лп", "Стойкость_лп"], axis=1)

In [633]:
df = df.rename(columns={"Покрытие": "id",
                        "Группа_лп": "paintwork_group",
                        "Наим_лп": "name",
                        "Кол_сл_лп": "paintwork_num_of_layers",
                        "Группа_гр": "primer_group",
                        "Можно_лп": "can_be_painted",
                        "Приоритет": "priority"
                       })

In [634]:
cursor.execute(open("sql_server/17.sql", "r").read())
conn.commit()

In [635]:
execute_values(df, "corr_prot_coatings")

execute_values() done


<h1>Виды конструкций</h1>

In [323]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Спр_видов_к', db_conn)

In [324]:
df["Имя_к"] = df["Имя_к"].str.strip()

In [325]:
df = df[df["Имя_к"].isna() == False]

In [326]:
df = df.rename(columns={"Шифр_к": "id",
                        "Имя_к": "name"
                       })

In [327]:
df.loc[df["id"] == 2, "name"] = "Двухветвевые колонны одноэтажных производственных зданий и крановых эстакад"
df.loc[df["id"] == 9, "name"] = "Подкраново-подстропильные фермы"
df.loc[df["id"] == 34, "name"] = "Переплеты, витражи из профильных труб с деталями крепления"

In [328]:
cursor.execute(open("sql_server/22.sql", "r").read())
conn.commit()

In [329]:
execute_values(df, "construction_types")

<h1>Подвиды конструкций</h1>

In [330]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Спр_видов_пк', db_conn)

In [331]:
df["Имя_пк"] = df["Имя_пк"].str.strip()
df["Расценка"] = df["Расценка"].str.strip()

In [332]:
df = df.drop_duplicates()

In [333]:
df = df.drop(["Шифр_пк"], axis=1)

In [334]:
df = df.rename(columns={"Поз_пк": "id",
                        "Шифр_к": "type_id",
                        "Имя_пк": "name",
                        "Расценка": "valuation"
                       })

In [335]:
df.loc[df["id"] == 11, "name"] = "высотой свыше 150 м, поставляемые отдельными элементами"
df.loc[df["id"] == 14, "name"] = "коробчатого сечения пролетом 12 м и более"
df.loc[df["id"] == 33, "name"] = "с шагом стропильных ферм 6 м"
df.loc[df["id"] == 34, "name"] = "с шагом стропильных ферм 12 м"
df.loc[df["id"] == 49, "name"] = "составного сечения из горячекатаных профилей, соединенных планками"
df.loc[df["id"] == 51, "name"] = "ветви из двутавров с парал. гранями полок, решетка из г/кат. уголков и швеллеров"
df.loc[df["id"] == 52, "name"] = "ветви из двутавров (ГОСТ 8239-72) или швеллеров и решетка из г/кат. уголков"
df.loc[df["id"] == 69, "name"] = "с поясами из тавров, стенкой из листовой стали, пролетом 6 м, массой 1 м,т:"
df.loc[df["id"] == 70, "name"] = "с поясами из тавров, стенкой из листовой стали, пролетом 12 м и более,т:"
df.loc[df["id"] == 71, "name"] = "из листовой стали пролетом 6 м, массой 1 м,т:"
df.loc[df["id"] == 72, "name"] = "из листовой стали пролетом 12 м, массой 1 м,т:"
df.loc[df["id"] == 83, "name"] = "пролетом 12 м, решетчатые из г/кат. и гнутых профилей, масса 1 м до 0.4 т"
df.loc[df["id"] == 84, "name"] = "пролетом 12 м, решетчатые из г/кат. и гнутых профилей, масса 1 м свыше 0.4 т"
df.loc[df["id"] == 85, "name"] = "пролетом 12 м, решетчатые сост. сечения из листовой стали, масса 1 м до 0.4 т"
df.loc[df["id"] == 86, "name"] = "пролетом 12 м, решетчатые сост. сечения из листовой стали, масса 1 м свыше 0.4 т"
df.loc[df["id"] == 87, "name"] = "составного сечения из листовой стали, масса 1 м,т:"
df.loc[df["id"] == 88, "name"] = "составного сечения из двух и более уголков или швеллеров, масса 1 м,т:"
df.loc[df["id"] == 89, "name"] = "из двутавров с параллельными гранями полок, масса 1 м,т:"
df.loc[df["id"] == 90, "name"] = "составного сечения из листовой стали с верхним опиранием настила, масса марки,т:"
df.loc[df["id"] == 91, "name"] = "составного сечения при пониженном опирании настила, масса марки,т:"
df.loc[df["id"] == 92, "name"] = "из двутавров с парал. гранями полок с верхним опиранием настила, масса до 1 т"
df.loc[df["id"] == 93, "name"] = "из двутавров с парал. гранями полок с верхним опиранием настила, масса свыше 1 т"
df.loc[df["id"] == 94, "name"] = "из двутавров с парал. гранями полок с пониж. опиранием настила, масса до 1 т"
df.loc[df["id"] == 95, "name"] = "из двутавров с парал. гранями полок с пониж. опиранием настила, масса свыше 1 т"
df.loc[df["id"] == 113, "name"] = "фермы из парных уголков и тавров, попер. балки из двутавров, масса 1 м до 0.5 т"
df.loc[df["id"] == 114, "name"] = "фермы из парных уголков и тавров, попер. балки из двутавров, масса 1 м до 1 т"
df.loc[df["id"] == 115, "name"] = "фермы из парных уголков и тавров, попер. балки из двутавров, масса 1 м свыше 1 т"

In [336]:
cursor.execute(open("sql_server/24.sql", "r").read())
conn.commit()

In [337]:
execute_values(df, "construction_subtypes")

<h1>Типовые наименования листов</h1>

In [338]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Спр_листов_н', db_conn)

In [339]:
df["ш_листа"] = df["ш_листа"].str.strip()

In [340]:
df = df.rename(columns={"ш_номер": "id",
                        "ш_листа": "name"
                       })

In [341]:
cursor.execute(open("sql_server/27.sql", "r").read())
conn.commit()

In [342]:
execute_values(df, "sheet_names")

<h1>Виды ссылочных документов</h1>

In [343]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Спр_ссыл_в', db_conn)

In [344]:
df["Наим_вида"] = df["Наим_вида"].str.strip()
df["Наим_вида"] = df["Наим_вида"].str.capitalize()

In [345]:
df = df.drop_duplicates(subset=["Наим_вида"])

In [346]:
df = df.rename(columns={"Вид_сд": "id",
                        "Наим_вида": "name"
                       })

In [347]:
cursor.execute(open("sql_server/40.sql", "r").read())
conn.commit()

In [348]:
execute_values(df, "linked_doc_types")

<h1>Ссылочные документы</h1>

In [631]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Спр_ссыл', db_conn)

In [632]:
df = df.replace("", np.nan)

In [633]:
df = df.dropna()

In [634]:
df = df.drop_duplicates(subset=["Шифр_сд"])

In [635]:
df = df.reset_index(drop=True)

In [636]:
df["Шифр_сд"] = df["Шифр_сд"].str.strip()
df["Обозн_сд"] = df["Обозн_сд"].str.strip()
df["Наим_сд"] = df["Наим_сд"].str.strip()

In [637]:
for i in range(0, len(df)):
    arr = df["Наим_сд"][i].split()
    new_str = ' '.join(arr)
    df.at[i, "Наим_сд"] = new_str

In [638]:
df["Наим_сд"] = df["Наим_сд"].str.replace("возду ха", "воздуха")
df["Наим_сд"] = df["Наим_сд"].str.replace("Материриалы", "Материалы")
df["Наим_сд"] = df["Наим_сд"].str.replace("рекоментации", "рекомендации")
df["Наим_сд"] = df["Наим_сд"].str.replace("рас пашные", "распашные")
df["Наим_сд"] = df["Наим_сд"].str.replace("металличе ческих", "металлических")
df["Наим_сд"] = df["Наим_сд"].str.replace("металличе ских", "металлических")
df["Наим_сд"] = df["Наим_сд"].str.replace("электри ческие", "электрические")
df["Наим_сд"] = df["Наим_сд"].str.replace("кра ны", "краны")
df["Наим_сд"] = df["Наим_сд"].str.replace("примене нием", "применением")
df["Наим_сд"] = df["Наим_сд"].str.replace("обслуживани сосудов", "обслуживания сосудов")
df["Наим_сд"] = df["Наим_сд"].str.replace("пе реплетами", "переплетами")
df["Наим_сд"] = df["Наим_сд"].str.replace("за полнением", "заполнением")
df["Наим_сд"] = df["Наим_сд"].str.replace("линй", "линий")
df["Наим_сд"] = df["Наим_сд"].str.replace("мосто вые", "мостовые")
df["Наим_сд"] = df["Наим_сд"].str.replace("покрыти ем", "покрытием")
df["Наим_сд"] = df["Наим_сд"].str.replace("со кращенному", "сокращенному")
df["Наим_сд"] = df["Наим_сд"].str.replace("покрытияпролетами", "покрытия пролетами")
df["Наим_сд"] = df["Наим_сд"].str.replace("широкополочныых", "широкополочных")
df["Наим_сд"] = df["Наим_сд"].str.replace("тав ров", "тавров")
df["Наим_сд"] = df["Наим_сд"].str.replace("зда ний", "зданий")
df["Наим_сд"] = df["Наим_сд"].str.replace("ремонтно -монтажных", "ремонтно-монтажных")
df["Наим_сд"] = df["Наим_сд"].str.replace("пок рытия", "покрытия")
df["Наим_сд"] = df["Наим_сд"].str.replace("профилирован ным", "профилированным")
df["Наим_сд"] = df["Наим_сд"].str.replace("стеклои минераловатных", "стекло и минераловатных")
df["Наим_сд"] = df["Наим_сд"].str.replace("окон,дверей", "окон, дверей")
df["Наим_сд"] = df["Наим_сд"].str.replace("пром.зданий", "пром. зданий")
df["Наим_сд"] = df["Наим_сд"].str.replace("строитель ные", "строительные")
df["Наим_сд"] = df["Наим_сд"].str.replace("сталь ные", "стальные")
df["Наим_сд"] = df["Наим_сд"].str.replace("наруж ного", "наружного")
df["Наим_сд"] = df["Наим_сд"].str.replace("ветров.район.", "ветров. район.")
df["Наим_сд"] = df["Наим_сд"].str.replace("выше и ниже -40 град.с.  ше и ниже -40 град.с.", "выше -40 град.с. и ниже -40 град.с.")
df["Наим_сд"] = df["Наим_сд"].str.replace("алю миниевых", "алюминиевых")
df["Наим_сд"] = df["Наим_сд"].str.replace("зданий,промышленных", "зданий, промышленных")
df["Наим_сд"] = df["Наим_сд"].str.replace("предприятий.шарнирные", "предприятий. Шарнирные")
df["Наим_сд"] = df["Наим_сд"].str.replace("связей.Чертежи", "связей. Чертежи")
df["Наим_сд"] = df["Наим_сд"].str.replace("сталь ными", "стальными")
df["Наим_сд"] = df["Наим_сд"].str.replace("унифици рованных", "унифицированных")
df["Наим_сд"] = df["Наим_сд"].str.replace("уз лы", "узлы")
df["Наим_сд"] = df["Наим_сд"].str.replace("железобе тонных", "железобетонных")
df["Наим_сд"] = df["Наим_сд"].str.replace("метал лопроката", "металлопроката")
df["Наим_сд"] = df["Наим_сд"].str.replace("выпу ску", "выпуску")
df["Наим_сд"] = df["Наим_сд"].str.replace("откры вающиеся", "открывающиеся")
df["Наим_сд"] = df["Наим_сд"].str.replace("светоаэра ционных", "светоаэрационных")
df["Наим_сд"] = df["Наим_сд"].str.replace("проектирования.Рабочие", "проектирования. Рабочие")
df["Наим_сд"] = df["Наим_сд"].str.replace("черте жи", "чертежи")
df["Наим_сд"] = df["Наим_сд"].str.replace("опорн.кранами", "опорн. кранами")
df["Наим_сд"] = df["Наим_сд"].str.replace("опорынми кранами", "опорными кранами")
df["Наим_сд"] = df["Наим_сд"].str.replace("несущ.конструкций", "несущ. конструкций")
df["Наим_сд"] = df["Наим_сд"].str.replace("подв.кранами", "подв. кранами")
df["Наим_сд"] = df["Наим_сд"].str.replace("кровлей,оборудованных", "кровлей, оборудованных")
df["Наим_сд"] = df["Наим_сд"].str.replace("одноэт.произв", "одноэт. произв")
df["Наим_сд"] = df["Наим_сд"].str.replace("произв.зданий", "произв. зданий")
df["Наим_сд"] = df["Наим_сд"].str.replace("кровлей,оборудованных", "кровлей, оборудованных")
df["Наим_сд"] = df["Наим_сд"].str.replace("специальн.назначения", "специальн. назначения")
df["Наим_сд"] = df["Наим_сд"].str.replace("зданий,оборудованных", "зданий, оборудованных")
df["Наим_сд"] = df["Наим_сд"].str.replace("две рей", "дверей")

In [639]:
df = df.drop_duplicates(subset=["Наим_сд"])

In [640]:
df.loc[df["Наим_сд"].str.endswith('.') == False, "Наим_сд"] = df[df["Наим_сд"].str.endswith('.') == False]["Наим_сд"] + '.'

In [641]:
df.insert(0, 'id', range(1, len(df) + 1))

In [642]:
df = df.rename(columns={"Шифр_сд": "code",
                        "Вид_сд": "type_id",
                        "Обозн_сд": "designation",
                        "Наим_сд": "name"
                       })

In [643]:
cursor.execute(open("sql_server/39.sql", "r").read())
conn.commit()

In [644]:
execute_values(df, "linked_docs")

insert into dbo.LinkedDocs(Id,Code,TypeId,Designation,Name) values (?, ?, ?, ?, ?)
1 БК1 24.0 3.010-1 вып. 1 Баки для хранения концентрированного и водного раствора пенообразователя емкостью от 6 до 80 м3. Бак V= 6м3.
insert into dbo.LinkedDocs(Id,Code,TypeId,Designation,Name) values (?, ?, ?, ?, ?)
2 БК2 24.0 3.010-1 вып. 2 Баки для хранения концентрированного и водного раствора пенообразователя емкостью от 6 до 80 м3. Бак V= 8м3.
insert into dbo.LinkedDocs(Id,Code,TypeId,Designation,Name) values (?, ?, ?, ?, ?)
3 БК3 24.0 3.010-1 вып. 3 Баки для хранения концентрированного и водного раствора пенообразователя емкостью от 6 до 80 м3. Бак V= 20м3.
insert into dbo.LinkedDocs(Id,Code,TypeId,Designation,Name) values (?, ?, ?, ?, ?)
4 БК4 24.0 3.010-1 вып. 4 Баки для хранения концентрированного и водного раствора пенообразователя емкостью от 6 до 80 м3. Бак V= 40м3.
insert into dbo.LinkedDocs(Id,Code,TypeId,Designation,Name) values (?, ?, ?, ?, ?)
5 БК5 24.0 3.010-1 вып. 5 Баки для хранения

insert into dbo.LinkedDocs(Id,Code,TypeId,Designation,Name) values (?, ?, ?, ?, ?)
134 ПБ1 8.0 7-Э10-80/85 Стальные подкрановые балки путей подвесного транспорта пролетом 12 м, грузоподъемностью до 5 т. Чертежи КМ.
insert into dbo.LinkedDocs(Id,Code,TypeId,Designation,Name) values (?, ?, ?, ?, ?)
135 ПБ10 8.0 1.426.2-3 вып. 8 Стальные подкрановые балки. разрезные подкрановые балки пролетами 6,12 и 18 м под мостовые электричес. краны общего назначения грузоподъемностью до 500 т с учетом технологических нагрузок. Чертежи КМ.
insert into dbo.LinkedDocs(Id,Code,TypeId,Designation,Name) values (?, ?, ?, ?, ?)
136 ПБ11 8.0 1.426.2-7 вып. 1 Балки подкрановые стальные под мостовые опорные краны. балки пролетами 6 и 12 м раз резные под однобалочные краны грузоподъемностью до 5 т. Чертежи КМ.
insert into dbo.LinkedDocs(Id,Code,TypeId,Designation,Name) values (?, ?, ?, ?, ?)
137 ПБ12 8.0 1.462-8 вып. 1 Стальные подкрановые балки под краны специального назначения для об'ектов черной металлургии. р

insert into dbo.LinkedDocs(Id,Code,TypeId,Designation,Name) values (?, ?, ?, ?, ?)
281 ФН16 14.0 1.464.3-19 Стальные конструкции аэрационных фонарей для производственных зданий. Чертежи КМ.
insert into dbo.LinkedDocs(Id,Code,TypeId,Designation,Name) values (?, ?, ?, ?, ?)
282 ФН17 14.0 1.464.3-20 вып. 0 Фонари зенитные глухие и открывающиеся с применением гнутых профилей из тонколистовой стали. материалы для проектирования. Рабочие чертежи узлов.
insert into dbo.LinkedDocs(Id,Code,TypeId,Designation,Name) values (?, ?, ?, ?, ?)
283 ФН18 14.0 1.464.3-20 вып. 1 Фонари зенитные глухие и открывающиеся с применением гнутых профилей из тонколистовой стали. Рабочие чертежи.
insert into dbo.LinkedDocs(Id,Code,TypeId,Designation,Name) values (?, ?, ?, ?, ?)
284 ФН19 14.0 1.464.2-21 вып. 1 Фонари зенитные с размерами светового проема 1,5 1,7 м со стальными переплетами. Фонарь зенитный глухой с трехслойным стеклопакетом. Рабочие чертежи.
insert into dbo.LinkedDocs(Id,Code,TypeId,Designation,Name)

<h1>Категории производства по пожарной и взрывопожарной опасности</h1>

In [591]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Категории_пр', db_conn)

In [592]:
df = df.replace("", np.nan)

In [593]:
df = df.rename(columns={"поз": "id",
                        "Категория": "category",
                        "Название": "name",
                        "Текст_од": "description"
                       })

In [594]:
df = df.where(pd.notnull(df), None)

In [595]:
cursor.execute(open("sql_server/5.sql", "r").read())
conn.commit()

In [596]:
execute_values(df, "fire_hazard_categories")

<h1>Проекты</h1>

In [645]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Марки', db_conn)
df.shape

(4112, 33)

In [646]:
df = df.dropna(subset=["код_марки", "обозн", "комплекс", "объект"])

In [647]:
e_ids = pd.read_csv("employee_ids.csv")
e_ids = e_ids.drop(["Unnamed: 0"], axis=1)
e_ids = e_ids.values.flatten().tolist()
df.loc[df["гл_спец"].isin(e_ids) == False, "гл_спец"] = None
df.loc[df["рук_гр"].isin(e_ids) == False, "рук_гр"] = None
df.loc[df["н_контр"].isin(e_ids) == False, "н_контр"] = None

In [648]:
df = df[df["обозн"].str.contains("демонтаж") == False]
df = df[df["обозн"].str.contains("Мновый") == False]
df["обозн"] = df["обозн"].str.replace(",", ".")

In [649]:
df2 = pd.DataFrame(df["обозн"].str.split('-КМ', 1).tolist(),
                                 columns = ['first','code'])

In [650]:
df = df.reset_index()
df2 = df2.join(df)

In [651]:
df2 = df2.dropna(subset=["code"])

In [652]:
df2["code"] = "КМ" + df2["code"]

In [653]:
df3 = pd.DataFrame(df2["first"].str.split('.', 1).tolist(),
                                 columns = ['baseSeries','middle'])

In [654]:
df2 = df2.reset_index()
df3 = df3.reset_index()
df4 = df2.join(df3[["baseSeries", "middle"]])

In [655]:
df4 = df4.drop(["first", "index"], axis=1)
df4 = df4.rename(columns={"код_марки": "id"})

In [656]:
df5 = pd.DataFrame(df4["middle"].fillna("").str.split('.', 1).tolist(),
                                 columns = ['node','subnode'])

In [657]:
df6 = pd.DataFrame(df5["subnode"].fillna("").str.split('-', 1).tolist(),
                                 columns = ['subnode_part','node_part'])

In [658]:
df5 = df5.reset_index()
df6 = df6.reset_index()
df7 = df5.join(df6[["subnode_part", "node_part"]])

In [659]:
df7["node_part"] = "-" + df7["node_part"]
df7["node2"] = df7["node"] + df7["node_part"].fillna("")

In [660]:
df4 = df4.reset_index(drop=True)
df7 = df7.reset_index(drop=True)
df8 = df4.join(df7[["node2", "subnode_part"]])

In [661]:
df8 = df8.rename(columns={"node2": "nodeCode", "subnode_part": "subnodeCode"})
df8 = df8.drop(["middle"], axis=1)

In [662]:
enc = OrdinalEncoder()
res = enc.fit_transform(np.array(df8["baseSeries"]).reshape(-1, 1))

In [663]:
s = pd.DataFrame(pd.Series(res.flatten()) + 1)
s = s.rename(columns={0: "project_id"})

In [664]:
df9 = df8.join(s)

In [665]:
projects_df = df9[["baseSeries", "комплекс", "project_id"]]

In [666]:
projects_df = projects_df.drop_duplicates(subset=["project_id"])

In [667]:
projects_df = projects_df.rename(columns={"baseSeries": "base_series", "комплекс": "name", "project_id": "id"})

In [668]:
cursor.execute(open("sql_server/xx_projects.sql", "r").read())
conn.commit()

In [669]:
execute_values(projects_df, "projects")

insert into dbo.Projects(BaseSeries,Name,Id) values (?, ?, ?)
М32636 ОАО "ММК". ЦЭСиП. ЦЭС. Реконструкция сети 110/10 кВ в 2013 году 16.0
insert into dbo.Projects(BaseSeries,Name,Id) values (?, ?, ?)
М32714-2  ПАО "ММК". Доменный цех. Печь №7 23.0
insert into dbo.Projects(BaseSeries,Name,Id) values (?, ?, ?)
М32839 ПАО "ММК". Доменный цех. Печь №1. Аспирация литейных дворов 37.0
insert into dbo.Projects(BaseSeries,Name,Id) values (?, ?, ?)
М32622 ОАО "ММК". ТЭЦ. Техническое перевооружение 14.0
insert into dbo.Projects(BaseSeries,Name,Id) values (?, ?, ?)
М60215 ОАО "ММК". Доменный цех. Печь №4. Модернизация 73.0
insert into dbo.Projects(BaseSeries,Name,Id) values (?, ?, ?)
М32715 ОАО"ММК". Доменный цех. Печь №9. Литейные дворы. Техперевооружение 24.0
insert into dbo.Projects(BaseSeries,Name,Id) values (?, ?, ?)
М32714-1 ОАО "ММК". Доменный цех. Печь №7 22.0
insert into dbo.Projects(BaseSeries,Name,Id) values (?, ?, ?)
М32710 ОАО "ММК". ТЭЦ 21.0
insert into dbo.Projects(BaseSeries,Name,

<h1>Узлы</h1>

In [670]:
df10 = pd.DataFrame(df9["объект"].str.split('.', 1).tolist(),
                                 columns = ['f_p','o_p'])

In [671]:
df10 = df10.fillna("")

In [672]:
df11 = df9.join(df10)

In [673]:
df11["node_feature"] = df11["baseSeries"] + df11["nodeCode"]

In [674]:
enc = OrdinalEncoder()
res = enc.fit_transform(np.array(df11["node_feature"]).reshape(-1, 1))

In [675]:
s = pd.DataFrame(pd.Series(res.flatten()) + 1)
s = s.rename(columns={0: "node_id"})

In [676]:
df12 = df11.join(s)

In [677]:
nodes_df = df12[["node_id", "f_p", "project_id", "nodeCode"]]

In [678]:
chiefEngineerIds = [173, 337]

In [679]:
nodes_df["chief_engineer"] = random.choices(chiefEngineerIds, k=len(nodes_df))

<ipython-input-679-24cbdb19fe02>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nodes_df["chief_engineer"] = random.choices(chiefEngineerIds, k=len(nodes_df))


In [680]:
nodes_df = nodes_df.rename(columns={"node_id": "id", "f_p": "name", "nodeCode": "code", "chief_engineer": "chief_engineer_id"})

In [681]:
nodes_df = nodes_df.drop_duplicates(subset=["id"])

In [684]:
cursor.execute(open("sql_server/xx_nodes.sql", "r").read())
conn.commit()

In [685]:
execute_values(nodes_df, "nodes")

insert into dbo.Nodes(Id,Name,Projectid,Code,Chiefengineerid) values (?, ?, ?, ?, ?)
26.0 ЦЭС 16.0 03 337
insert into dbo.Nodes(Id,Name,Projectid,Code,Chiefengineerid) values (?, ?, ?, ?, ?)
34.0 Воздухонагреватель №23-бис 23.0 07 173
insert into dbo.Nodes(Id,Name,Projectid,Code,Chiefengineerid) values (?, ?, ?, ?, ?)
69.0 Аспирационная установка литейных дворов 37.0 05 337
insert into dbo.Nodes(Id,Name,Projectid,Code,Chiefengineerid) values (?, ?, ?, ?, ?)
24.0 Котел ТП-10 14.0  173
insert into dbo.Nodes(Id,Name,Projectid,Code,Chiefengineerid) values (?, ?, ?, ?, ?)
145.0 Воздухонагреватели 73.0 04 337
insert into dbo.Nodes(Id,Name,Projectid,Code,Chiefengineerid) values (?, ?, ?, ?, ?)
35.0 Реконструкция площадок 24.0 09 337
insert into dbo.Nodes(Id,Name,Projectid,Code,Chiefengineerid) values (?, ?, ?, ?, ?)
33.0 Воздухонагреватель №23-бис 22.0 07 173
insert into dbo.Nodes(Id,Name,Projectid,Code,Chiefengineerid) values (?, ?, ?, ?, ?)
32.0 Котел ТП-10 21.0  173
insert into dbo.Nodes(I

insert into dbo.Nodes(Id,Name,Projectid,Code,Chiefengineerid) values (?, ?, ?, ?, ?)
159.0 Система измерения температуры газа по радиусу бесконтактным способом 87.0  337
insert into dbo.Nodes(Id,Name,Projectid,Code,Chiefengineerid) values (?, ?, ?, ?, ?)
22.0 Внешнее электроснабжение 13.0 14 337
insert into dbo.Nodes(Id,Name,Projectid,Code,Chiefengineerid) values (?, ?, ?, ?, ?)
113.0 Электропомещение ЭП-3 54.0 04 173
insert into dbo.Nodes(Id,Name,Projectid,Code,Chiefengineerid) values (?, ?, ?, ?, ?)
174.0 Видеонаблюдение на рабочих площадках 102.0  337
insert into dbo.Nodes(Id,Name,Projectid,Code,Chiefengineerid) values (?, ?, ?, ?, ?)
43.0 Весы железнодорожные 28.0 10 337
insert into dbo.Nodes(Id,Name,Projectid,Code,Chiefengineerid) values (?, ?, ?, ?, ?)
111.0  "Грязный" оборотный цикл водоснабжения печей и рабочих клетей 52.0 26 173
insert into dbo.Nodes(Id,Name,Projectid,Code,Chiefengineerid) values (?, ?, ?, ?, ?)
109.0 Модернизация системы гидросбива окалины стана 2000 г 50.0  

<h1>Подузлы</h1>

In [686]:
df13 = pd.DataFrame(df12["o_p"].str.split('.', 1).tolist(),
                                 columns = ['s_p','m_p'])

In [687]:
df13 = df13.fillna("")

In [688]:
df13["s_p"] = df13["s_p"].str.strip()
df13["m_p"] = df13["m_p"].str.strip()

In [689]:
df14 = df12.join(df13)

In [690]:
df14["subnode_feature"] = df11["node_feature"] + df11["subnodeCode"]

In [691]:
enc = OrdinalEncoder()
res = enc.fit_transform(np.array(df14["subnode_feature"]).reshape(-1, 1))

In [692]:
s = pd.DataFrame(pd.Series(res.flatten()) + 1)
s = s.rename(columns={0: "subnode_id"})

In [693]:
df15 = df14.join(s)

In [694]:
subnodes_df = df15[["subnode_id", "node_id", "subnodeCode", "s_p"]]

In [695]:
subnodes_df = subnodes_df.rename(columns={"subnode_id": "id", "s_p": "name", "subnodeCode": "code"})

In [696]:
subnodes_df = subnodes_df.drop_duplicates(subset=["id"])

In [697]:
cursor.execute(open("sql_server/xx_subnodes.sql", "r").read())
conn.commit()

In [698]:
execute_values(subnodes_df, "subnodes")

insert into dbo.Subnodes(Id,NodeId,Code,Name) values (?, ?, ?, ?)
26.0 26.0  ГРУ-10,5 кВ
insert into dbo.Subnodes(Id,NodeId,Code,Name) values (?, ?, ?, ?)
34.0 34.0 03 1 и 2 этапы
insert into dbo.Subnodes(Id,NodeId,Code,Name) values (?, ?, ?, ?)
77.0 69.0  Каркас здания
insert into dbo.Subnodes(Id,NodeId,Code,Name) values (?, ?, ?, ?)
24.0 24.0  Установка электрофильтра с системой удаления золы
insert into dbo.Subnodes(Id,NodeId,Code,Name) values (?, ?, ?, ?)
172.0 145.0 03 Станция разрежения
insert into dbo.Subnodes(Id,NodeId,Code,Name) values (?, ?, ?, ?)
171.0 145.0 01 Ремонт шихтового бункера БЗУ
insert into dbo.Subnodes(Id,NodeId,Code,Name) values (?, ?, ?, ?)
35.0 35.0 04 Балки рабочей площадки в осях 10… 15
insert into dbo.Subnodes(Id,NodeId,Code,Name) values (?, ?, ?, ?)
33.0 33.0 03 1 и 2 этапы
insert into dbo.Subnodes(Id,NodeId,Code,Name) values (?, ?, ?, ?)
175.0 145.0 07 Замена горловин бункеров агломерата
insert into dbo.Subnodes(Id,NodeId,Code,Name) values (?, ?, ?, ?)
32

insert into dbo.Subnodes(Id,NodeId,Code,Name) values (?, ?, ?, ?)
198.0 166.0  Видеонаблюдение
insert into dbo.Subnodes(Id,NodeId,Code,Name) values (?, ?, ?, ?)
105.0 94.0  Козырек и рама под шкаф
insert into dbo.Subnodes(Id,NodeId,Code,Name) values (?, ?, ?, ?)
151.0 128.0  1К675
insert into dbo.Subnodes(Id,NodeId,Code,Name) values (?, ?, ?, ?)
17.0 17.0  Вставка ремонтная с холодильником заплечиков
insert into dbo.Subnodes(Id,NodeId,Code,Name) values (?, ?, ?, ?)
88.0 80.0  Внутриплощадочные сети и сооружения
insert into dbo.Subnodes(Id,NodeId,Code,Name) values (?, ?, ?, ?)
23.0 23.0 02 "Чистый" оборотный цикл водоснабжения
insert into dbo.Subnodes(Id,NodeId,Code,Name) values (?, ?, ?, ?)
169.0 144.0  Опоры трубопроводов
insert into dbo.Subnodes(Id,NodeId,Code,Name) values (?, ?, ?, ?)
199.0 167.0  Свеча дожигания конвертера №3
insert into dbo.Subnodes(Id,NodeId,Code,Name) values (?, ?, ?, ?)
37.0 38.0  Балки под рельсы
insert into dbo.Subnodes(Id,NodeId,Code,Name) values (?, ?, ?, ?

<h1>Марки</h1>

In [699]:
df15 = df15.drop(["обозн", "комплекс", "объект", "level_0", "ОЗ_марки"], axis=1)

In [700]:
df15["t_эксплуат"] = df15["t_эксплуат"].str.replace("минус ", "-")
df15["t_эксплуат"] = df15["t_эксплуат"].str.replace("плюс ", "")
df15["t_эксплуат"] = df15["t_эксплуат"].str.replace("+", "")
df15["t_эксплуат"] = df15["t_эксплуат"].str.replace("- 39", "-39")
df15 = df15[df15["t_эксплуат"] != "-"]
df15 = df15[df15["t_эксплуат"] != "минус"]
df15["t_эксплуат"] = df15["t_эксплуат"].astype("float64")

In [701]:
df15["об_марки"] = df15["об_марки"].str.strip()
df15["об_марки"] = df15["об_марки"].str.replace("KM", "КМ")
df15 = df15[df15["об_марки"] != "по"]

In [702]:
df15 = df15.drop(["об_марки"], axis=1)

In [703]:
df15 = df15.drop(["project_id", "f_p", "o_p", "s_p", "subnode_feature", "node_feature", "subnodeCode"], axis=1)

In [750]:
df15 = df15.drop(["baseSeries", "nodeCode", "node_id"], axis=1)

In [751]:
df15["прим"] = df15["прим"].str.strip()
df15["прим"] = df15["прим"].str.replace("\r\n       ", " ")

In [752]:
df15 = df15.drop(["ДопКод"], axis=1)

In [753]:
df15["категория_пр"] = 3

In [754]:
df15["агрессивность"] += 1
df15["группа_газов"] += 1
df15["материал"] += 1
df15["вп_болты"] += 1

# Part
df15.loc[df15["тип_ЛКМ"] == "ПФ", "тип_ЛКМ"] = 1
df15.loc[df15["тип_ЛКМ"] == "АУ", "тип_ЛКМ"] = 2
df15.loc[df15["тип_ЛКМ"] == "ЭЭ", "тип_ЛКМ"] = 3
df15.loc[df15["тип_ЛКМ"] == "МА", "тип_ЛКМ"] = 4
df15.loc[df15["тип_ЛКМ"] == "ПВ", "тип_ЛКМ"] = 11

In [755]:
df15 = df15.drop_duplicates(subset=["code", "subnode_id"])

In [756]:
df15_operating_cond = df15.copy()

In [757]:
df15 = df15.rename(columns={"код_отд": "department_id",
                            "подп1": "signed1_id",
                            "подп2": "signed2_id",
                            "гл_спец": "chief_specialist_id",
                            "рук_гр": "group_leader_id",
                            "н_контр": "main_builder_id",
                            "дата_выд": "issue_date",
                            "кол_томов": "num_of_volumes",
                            "дата_ред": "edited_date",
                            "коэф_надежн": "safety_coeff",
                            "t_эксплуат": "operating_temp",
                            "зона_эксплуат": "operating_zone",
                            "группа_газов": "gas_group",
                            "агрессивность": "aggressiveness",
                            "материал": "material",
                            "тип_ЛКМ": "paintwork_type",
                            "прим": "note",
                            "категория_пр": "fire_hazard_category_id",
                            "вп_болты": "high_tensile_bolts",
                            "п_транспорт": "p_transport",
                            "п_площадки": "p_site",
                            "10ХСНД": "xcnd",
                            "ТекстЗдСм": "task_text",
                            "ДопОбъемы": "additional_text",
                            "ВесПоВМП": "vmp_weight",
                            "ВыпЗдСм": "impl_3d_estimate",
                            "m_p": "name"
                       })

In [758]:
df15["issue_date"] = df15["issue_date"].astype(object).where(df15["issue_date"].notnull(), None)
df15["edited_date"] = df15["edited_date"].astype(object).where(df15["edited_date"].notnull(), None)
df15 = df15.where(pd.notnull(df15), None)

In [759]:
df15["chief_specialist_id"] = df15["chief_specialist_id"].replace(0, None)
df15["group_leader_id"] = df15["group_leader_id"].replace(0, None)
df15["main_builder_id"] = df15["main_builder_id"].replace(0, None)

In [760]:
df16 = df15.drop(["vmp_weight", "impl_3d_estimate", "xcnd", "task_text", "additional_text", "safety_coeff", "aggressiveness", "operating_temp", "operating_zone", "gas_group", "material", "high_tensile_bolts"], axis=1)

In [761]:
df16 = df16.reset_index(drop=True)

In [762]:
df16["id"].to_csv("mark_ids.csv")

In [763]:
cursor.execute(open("sql_server/7.sql", "r").read())
conn.commit()

In [764]:
execute_values(df16, "marks")

execute_values() done


<h1>Условия эксплуатации марки</h1>

In [766]:
df_op_cond = df15_operating_cond.loc[:, ["id", "коэф_надежн", "агрессивность", "t_эксплуат", "зона_эксплуат", "группа_газов", "материал", "тип_ЛКМ", "вп_болты"]]

In [767]:
df_op_cond["t_эксплуат"] = df_op_cond["t_эксплуат"].fillna(-34)

In [768]:
df_op_cond = df_op_cond.rename(columns={"id": "mark_id",
                            "коэф_надежн": "safety_coeff",
                            "t_эксплуат": "temperature",
                            "зона_эксплуат": "operating_area_id",
                            "группа_газов": "gas_group_id",
                            "агрессивность": "env_aggressiveness_id",
                            "материал": "construction_material_id",
                            "тип_ЛКМ": "paintwork_type_id",
                            "вп_болты": "high_tensile_bolts_type_id"
                       })

In [769]:
cursor.execute(open("sql_server/y_mark_operating_conditions.sql", "r").read())
conn.commit()

In [770]:
execute_values(df_op_cond, "mark_operating_conditions")

execute_values() done


<h1>Задание на смету</h1>

In [771]:
df_task = df15.loc[:, ["id", "task_text", "additional_text"]]

In [772]:
df_task["task_text"] = df_task["task_text"].str.strip()

In [773]:
df['обозн'] = 'Разработать сметную документацию к чертежам ' + df['обозн'].astype(str) + '\r\nСостав и объемы работ:'

In [774]:
df_task.loc[df_task["task_text"].isna(), "task_text"] = list(df.loc[df["код_марки"].isin(df_task.loc[df_task["task_text"].isna(), "id"]), "обозн"])

In [775]:
df_task = df_task.rename(columns={"id": "mark_id"})

In [776]:
df_task["task_text"] = df_task["task_text"].str.replace("\r\n", "\n")

In [777]:
df_task["additional_text"] = df_task["additional_text"].str.replace("\r\n", "\n")

In [778]:
cursor.execute(open("sql_server/y_estimate_task.sql", "r").read())
conn.commit()

In [779]:
execute_values(df_task, "estimate_task")

execute_values() done


<h1>Выпуски спецификаций</h1>

In [780]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.ВыпСпец', db_conn)

In [781]:
df = df.replace("", np.nan)

In [782]:
df = df.drop_duplicates(subset=["код_марки", "выпуск"])

In [783]:
df = df.drop(["вкл_вып"], axis=1)

In [784]:
df["текущий"] = df["текущий"].map(lambda x: True if (x == 1) else False)

In [785]:
df = df.rename(columns={"Поз_выпуска": "id",
                        "код_марки": "mark_id",
                        "выпуск": "num",
                        "дата_созд": "created_date",
                        "текущий": "is_current",
                        "прим": "note"
                       })

In [786]:
df["created_date"] = df["created_date"].astype(object).where(df["created_date"].notnull(), datetime(2000, 1, 1))
df = df.where(pd.notnull(df), None)

In [787]:
m_ids = pd.read_csv("mark_ids.csv")
m_ids = m_ids.drop(["Unnamed: 0"], axis=1)
m_ids = m_ids.values.flatten().tolist()

In [788]:
df = df[df["mark_id"].isin(m_ids)]

In [789]:
df.loc[:, ["id", "mark_id", "num"]].to_csv("specs.csv")

In [790]:
cursor.execute(open("sql_server/3.sql", "r").read())
conn.commit()

In [791]:
execute_values(df, "specifications")

execute_values() done


<h1>Листы</h1>

In [793]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Листы', db_conn)

In [794]:
df = df.replace("", np.nan)

In [795]:
df = df[df["номер"].isna() == False]

In [796]:
df["тип_док"] = df["тип_док"] + 1

In [797]:
df["вып"] = df["вып"].map(lambda x: np.nan if x == 0 else x)

In [798]:
df["пров"] = df["пров"].map(lambda x: np.nan if x == 0 else x)

In [799]:
df["норм"] = df["норм"].map(lambda x: np.nan if x == 0 else x)

In [800]:
df = df.drop(["шифр_вып", "шифр_пров", "шифр_норм"], axis=1)

In [801]:
df["формат"] = df["формат"].map(lambda x: 1.0 if np.isnan(x) else x)

In [802]:
df = df.drop_duplicates(subset=["код_марки", "номер", "тип_док"])

In [803]:
df = df[df["вып"].isna() == False]

In [804]:
df["название"] = df["название"].str.strip()

In [805]:
for i in range(0, len(df)):
    try:
        arr = df["название"][i].split()
        new_str = ' '.join(arr)
        df.at[i, "название"] = new_str
    except Exception:
        pass

In [806]:
df = df.rename(columns={"поз_листа": "id",
                        "код_марки": "mark_id",
                        "номер": "num",
                        "название": "name",
                        "формат": "form",
                        "вып": "creator_id",
                        "пров": "inspector_id",
                        "норм": "norm_contr_id",
                        "тип_док": "type_id",
                        "выпуск": "release_num",
                        "листов": "num_of_pages",
                        "прим": "note"
                       })

In [807]:
df = df.where(pd.notnull(df), None)

In [808]:
m_ids = pd.read_csv("mark_ids.csv")
m_ids = m_ids.drop(["Unnamed: 0"], axis=1)
m_ids = m_ids.values.flatten().tolist()

In [809]:
e_ids = pd.read_csv("employee_ids.csv")
e_ids = e_ids.drop(["Unnamed: 0"], axis=1)
e_ids = e_ids.values.flatten().tolist()

In [810]:
df = df[df["mark_id"].isin(m_ids)]
df.loc[df["creator_id"].isin(e_ids) == False, "creator_id"] = None
df.loc[df["inspector_id"].isin(e_ids) == False, "inspector_id"] = None
df.loc[df["norm_contr_id"].isin(e_ids) == False, "norm_contr_id"] = None

In [811]:
df["release_num"] = df["release_num"].fillna(0)
df["num_of_pages"] = df["num_of_pages"].fillna(0)

In [812]:
df = df[df["creator_id"].isna() == False]
df = df[df["name"].isna() == False]

In [813]:
cursor.execute(open("sql_server/6.sql", "r").read())
conn.commit()

In [814]:
execute_values(df, "docs")

execute_values() done


<h1>Обозначения марок</h1>

In [816]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Об_марок', db_conn)

In [817]:
df = df.iloc[:-1, :]

In [818]:
df.loc[0, "об_марки"] = "КМД"
df.loc[0, "наим_марки"] = "Конструкции металлические деталировочные"

In [819]:
df = df.rename(columns={"Поз": "id",
                        "об_марки": "designation",
                        "наим_марки": "name"
                       })

In [820]:
cursor.execute(open("sql_server/9.sql", "r").read())
conn.commit()

In [821]:
execute_values(df, "mark_designations")

execute_values() done


<h1>Прилагаемые документы марки</h1>

In [823]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Од_прил', db_conn)

In [824]:
df = df.replace("", np.nan)

In [825]:
df = df.drop_duplicates(subset=["код_марки", "обозн_прил"])

In [826]:
df = df.where(pd.notnull(df), None)

In [827]:
df = df.rename(columns={"Поз_прил": "id",
                        "код_марки": "mark_id",
                        "наим_прил": "name",
                        "прим": "note",
                        "обозн_прил": "designation"
                       })

In [828]:
m_ids = pd.read_csv("mark_ids.csv")
m_ids = m_ids.drop(["Unnamed: 0"], axis=1)
m_ids = m_ids.values.flatten().tolist()
df = df[df["mark_id"].isin(m_ids)]

In [829]:
cursor.execute(open("sql_server/12.sql", "r").read())
conn.commit()

In [830]:
execute_values(df, "attached_docs")

execute_values() done


<h1>Согласования</h1>

In [832]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Од_согл', db_conn)

In [833]:
df = df.replace("", np.nan)

In [834]:
df = df.drop(["отдел1", "отдел2", "отдел3", "отдел4", "отдел5", "отдел6", "отдел7"], axis=1)

In [835]:
p1 = df.iloc[:, [0,1]]
p1 = p1.rename(columns={"исп1": "исп"})

p2 = df.iloc[:, [0,2]]
p2 = p2.rename(columns={"исп2": "исп"})

p3 = df.iloc[:, [0,3]]
p3 = p3.rename(columns={"исп3": "исп"})

p4 = df.iloc[:, [0,4]]
p4 = p4.rename(columns={"исп4": "исп"})

p5 = df.iloc[:, [0,5]]
p5 = p5.rename(columns={"исп5": "исп"})

p6 = df.iloc[:, [0,6]]
p6 = p6.rename(columns={"исп6": "исп"})

p7 = df.iloc[:, [0,7]]
p7 = p7.rename(columns={"исп7": "исп"})

In [836]:
df = pd.concat([p1, p2, p3, p4, p5, p6, p7], ignore_index=True)

In [837]:
df = df.dropna()
df = df[df["исп"] != 0]

In [838]:
df = df.drop_duplicates()

In [839]:
df.insert(0, 'id', range(1, len(df) + 1))

In [840]:
df = df.rename(columns={"код_марки": "mark_id",
                        "исп": "employee_id"
                       })

In [841]:
m_ids = pd.read_csv("mark_ids.csv")
e_ids = pd.read_csv("employee_ids.csv")

In [842]:
m_ids = m_ids.drop(["Unnamed: 0"], axis=1)
e_ids = e_ids.drop(["Unnamed: 0"], axis=1)

In [843]:
m_ids = m_ids.values.flatten().tolist()
e_ids = e_ids.values.flatten().tolist()

In [844]:
df = df[df["mark_id"].isin(m_ids)]
df = df[df["employee_id"].isin(e_ids)]

In [845]:
cursor.execute(open("sql_server/13.sql", "r").read())
conn.commit()

In [846]:
execute_values(df, "mark_approvals")

execute_values() done


<h1>Ссылочные документы марки</h1>

In [848]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Од_ссыл', db_conn)

In [849]:
df = df.replace("", np.nan)

In [850]:
df= df[df["шифр_сд"] != "1"]
df["шифр_сд"] = df["шифр_сд"].str.upper()

In [851]:
m_ids = pd.read_csv("mark_ids.csv")
m_ids = m_ids.drop(["Unnamed: 0"], axis=1)
m_ids = m_ids.values.flatten().tolist()
df = df[df["код_марки"].isin(m_ids)]

In [852]:
for i in range(1, 301):
    df = df.append(pd.Series(), ignore_index=True)

<ipython-input-852-9c8014e24f43>:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df = df.append(pd.Series(), ignore_index=True)


In [853]:
df.loc[:, "код_марки"] = random.choices(m_ids, k=300)
df.loc[:, "шифр_сд"] = random.choices(range(1, 300), k=300)

In [854]:
df = df.drop_duplicates(subset=["код_марки", "шифр_сд"])

In [855]:
df = df.drop(["Поз_сд"], axis=1)

In [856]:
df.insert(0, 'id', range(1, len(df) + 1))

In [857]:
df = df.rename(columns={"код_марки": "mark_id",
                        "шифр_сд": "linked_doc_id",
                        "прим": "note"
                       })

In [858]:
cursor.execute(open("sql_server/14.sql", "r").read())
conn.commit()

In [859]:
execute_values(df, "mark_linked_docs")

execute_values() done


<h1>Учет дополнительных работ</h1>

In [861]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Од_вып_доп', db_conn)

In [862]:
df = df.replace("", np.nan)

In [863]:
df = df.drop(["шифр_исп"], axis=1)

In [864]:
df = df[df["код_исп"].isna() == False]

In [865]:
df = df.drop_duplicates(subset=["код_марки", "код_исп"])

In [866]:
m_ids = pd.read_csv("mark_ids.csv")
m_ids = m_ids.drop(["Unnamed: 0"], axis=1)
m_ids = m_ids.values.flatten().tolist()
df = df[df["код_марки"].isin(m_ids)]

In [867]:
df = df.rename(columns={"Поз_вып_доп": "id",
                        "код_марки": "mark_id",
                        "код_исп": "employee_id",
                        "расчет": "valuation",
                        "заказ": "metal_order",
                       })

In [868]:
cursor.execute(open("sql_server/11.sql", "r").read())
conn.commit()

In [869]:
execute_values(df, "additional_work")

execute_values() done


<h1>Типовые конструкции</h1>

In [871]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Тс_типовые', db_conn)

In [872]:
df = df[df["имя_тк"].isna() == False]

In [873]:
df["имя_тк"] = df["имя_тк"].str.strip()

<h2>Типовые наименования типовых конструкций</h2>

In [874]:
st_name_df = pd.DataFrame(data=df["имя_тк"].unique(), columns=["name"])

In [875]:
st_name_df.loc[3, "name"] = "Рельсы крановые КР-70 сталь К63"
st_name_df = st_name_df[st_name_df["name"] != "КР-70"]
st_name_df.loc[5, "name"] = "Рельсы крановые Р-43"
st_name_df = st_name_df[st_name_df["name"] != "Оцинкованная сталь ГОСТ14918-80*"]
st_name_df.loc[11, "name"] = "Рельсы крановые КР-70"
st_name_df.loc[12, "name"] = "Детали крепления крановых рельсов"
st_name_df = st_name_df[st_name_df["name"] != "Крановый рельс КР-70"]
st_name_df.loc[16, "name"] = "Накладка 5 на рельс Р-24 по ГОСТ 8141-56"

In [876]:
st_name_df.insert(0, 'id', range(1, len(st_name_df) + 1))

In [877]:
cursor.execute(open("sql_server/yyy_1.sql", "r").read())
conn.commit()

In [878]:
execute_values(st_name_df, "standard_construction_names")

execute_values() done


--------------------

In [879]:
m_ids = pd.read_csv("mark_ids.csv")
m_ids = m_ids.drop(["Unnamed: 0"], axis=1)
m_ids = m_ids.values.flatten().tolist()
df = df[df["код_марки"].isin(m_ids)]

In [880]:
specs = pd.read_csv("specs.csv")
specs = specs.drop(["Unnamed: 0"], axis=1)

In [881]:
for index, row in df.iterrows():
    try:
        df.loc[index, "specification_id"] = specs.loc[(specs["mark_id"] == row["код_марки"]) & (specs["num"] == row["выпуск"]), "id"].values[0]
    except:
        pass

In [882]:
df = df.drop(["код_марки", "выпуск"], axis=1)

In [883]:
df.loc[24, "кол_тк"] = 1

In [884]:
# https://wikiweight.info/strmat/massa-ankerov-hilti.html
df.loc[28, "вес_тк"] = 0.056
df.loc[32, "вес_тк"] = 0.056

In [885]:
df.loc[25, "вес_тк"] = 0.024
df.loc[29, "вес_тк"] = 0.024
df.loc[25, "кол_тк"] = 360
df.loc[29, "кол_тк"] = 360
df = df[df["сч_тк"] != 54]
df = df[df["сч_тк"] != 58]

In [886]:
df = df.rename(columns={"сч_тк": "id",
                        "имя_тк": "name",
                        "кол_тк": "num",
                        "лист_тк": "sheet",
                        "вес_тк": "weight"
                       })

In [887]:
df = df.where(pd.notnull(df), None)

In [888]:
cursor.execute(open("sql_server/47.sql", "r").read())

In [889]:
execute_values(df, "standard_constructions")

execute_values() done


<h1>Виды конструкций марки</h1>

In [891]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Тс_Виды_к', db_conn)

In [892]:
df = df.drop_duplicates()

In [893]:
df = df.dropna(subset=["кт_окраски", "кол_тип_констр", "имя_к", "Шифр_к", "Шифр_пк"])

In [894]:
m_ids = pd.read_csv("mark_ids.csv")
m_ids = m_ids.drop(["Unnamed: 0"], axis=1)
m_ids = m_ids.values.flatten().tolist()
df = df[df["код_марки"].isin(m_ids)]

In [895]:
specs = pd.read_csv("specs.csv")
specs = specs.drop(["Unnamed: 0"], axis=1)

In [896]:
df.insert(1, "specification_id", 0, True) 

In [897]:
for index, row in df.iterrows():
    try:
        df.loc[index, "specification_id"] = specs.loc[(specs["mark_id"] == row["код_марки"]) & (specs["num"] == row["выпуск"]), "id"].values[0]
    except:
        pass

In [898]:
df = df[df["specification_id"] != 0]

In [899]:
df = df.drop(["код_марки", "выпуск"], axis=1)

In [900]:
df["Шифр_пк"] = df["Шифр_пк"].replace(0, np.nan)

In [901]:
df = df.drop(["вкл", "окраска", "огнезащита", "ПределОС", "ПредСост_несущ", "ПредСост_целост", "ПредСост_тепло_изол", "ПредСост_тепло_поток", "ПредСост_газ", "статус_к", "ТеорРасходОЗ"], axis=1)

In [902]:
df = df.rename(columns={"сч_констр": "id",
                        "имя_к": "name",
                        "Шифр_к": "type_id",
                        "Шифр_пк": "subtype_id",
                        "расценка": "valuation",
                        "шифр_тип_альбома": "standard_album_code",
                        "кол_тип_констр": "num_of_standard_constructions",
                        "притупление": "has_edge_blunting",
                        "динамика": "has_dynamic_load",
                        "фланцы": "has_flanged_connections",
                        "контроль_сварки": "welding_control_id",
                        "кт_окраски": "paintwork_coeff",
                       })

In [903]:
df = df.where(pd.notnull(df), None)

In [904]:
df = df[df["type_id"] < 88]

In [905]:
df = df[df["type_id"] > 0]

In [906]:
df = df.drop_duplicates(subset=["specification_id", "name", "paintwork_coeff"])

In [907]:
df.loc[:, ["id"]].to_csv("construction_ids.csv")

In [908]:
cursor.execute(open("sql_server/46.sql", "r").read())
conn.commit()

In [909]:
execute_values(df, "constructions")

execute_values() done


<h1>Справочник болтов, диаметры</h1>

In [911]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Спр_болтов', db_conn)

In [912]:
df = df.rename(columns={"код_д": "id",
                        "диаметр": "diameter",
                        "масса_г": "nut_weight",
                        "сталь_ш": "washer_steel",
                        "масса_ш": "washer_weight",
                        "толщина_ш": "washer_thickness",
                        "ТУ_б": "bolt_tech_spec",
                        "кл_пр_б": "strength_class",
                        "ТУ_г": "nut_tech_spec",
                        "ТУ_ш": "washer_tech_spec",
                       })

In [913]:
cursor.execute(open("sql_server/20.sql", "r").read())
conn.commit()

In [914]:
execute_values(df, "bolt_diameters")

execute_values() done


<h1>Справочник болтов, длины</h1>

In [916]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Спр_болтов_дл', db_conn)

In [917]:
df = df.rename(columns={"Код_болта": "id",
                        "код_д": "diameter_id",
                        "длина_б": "length",
                        "длина_нар": "screw_length",
                        "масса_б": "weight"
                       })

In [918]:
cursor.execute(open("sql_server/21.sql", "r").read())
conn.commit()

In [919]:
execute_values(df, "bolt_lengths")

execute_values() done


<h1>Высокопрочные болты видов конструкций</h1>

In [921]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Тс_болты', db_conn)

In [922]:
df

,поз_болта,сч_констр,код_диаметра,пакет,кол,гаек,шайб
0,13,2573.0,NaN,0.0,0.0,1,2
1,14,4140.0,2.0,60.0,8.0,1,2
2,15,NaN,NaN,0.0,0.0,1,2
3,16,716.0,2.0,0.0,0.0,1,2
4,17,5262.0,NaN,0.0,0.0,1,2
5,18,7585.0,1.0,70.0,265.0,1,2
6,19,7758.0,NaN,NaN,NaN,1,2
7,20,7758.0,0.0,NaN,0.0,1,2
8,22,7896.0,2.0,72.0,77.0,1,2
9,24,958.0,1.0,25.0,50.0,1,2


In [923]:
c_ids = pd.read_csv("construction_ids.csv")
c_ids = c_ids.drop(["Unnamed: 0"], axis=1)
c_ids = c_ids.values.flatten().tolist()

In [924]:
df = df[df["сч_констр"].isin(c_ids)]

In [925]:
df.loc[:, "код_диаметра"] = df.loc[:, "код_диаметра"].replace(0, np.nan)

C:\Users\Public\Anaconda\lib\site-packages\pandas\core\indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [926]:
df = df.dropna()

In [927]:
df = df.rename(columns={"поз_болта": "id",
                        "сч_констр": "construction_id",
                        "код_диаметра": "diameter_id",
                        "пакет": "packet",
                        "кол": "num",
                        "гаек": "nut_num",
                        "шайб": "washer_num",
                       })

In [928]:
cursor.execute(open("sql_server/45.sql", "r").read())
conn.commit()

In [929]:
execute_values(df, "construction_bolts")

execute_values() done


<h1>Виды профилей</h1>

In [931]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Спр_видов_п', db_conn)

In [932]:
df["Имя_в"] = df["Имя_в"].str.strip()
df["Прим"] = df["Прим"].str.strip()

In [933]:
df = df.drop_duplicates(subset=["Имя_в"])

In [934]:
df = df.rename(columns={"Шифр_в": "id",
                        "Имя_в": "name",
                        "Прим": "note"
                       })

In [935]:
df = df.sort_values(by=["id"])

In [936]:
df = df.reset_index(drop=True)

In [937]:
df.loc[:, ["id"]].to_csv("prof.csv")

In [938]:
cursor.execute(open("sql_server/23.sql", "r").read())
conn.commit()

In [939]:
execute_values(df, "profile_classes")

execute_values() done


<h1>Типы профилей</h1>

In [941]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Спр_типов_проф', db_conn)

In [942]:
df["Имя_т"] = df["Имя_т"].str.strip()

In [943]:
df = df.rename(columns={"Тип": "id",
                        "Имя_т": "name"
                       })

In [944]:
cursor.execute(open("sql_server/42.sql", "r").read())
conn.commit()

In [945]:
execute_values(df, "profile_types")

execute_values() done


<h1>Профили</h1>

In [947]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Спр_проф_полн', db_conn)

In [948]:
df = df.replace("", np.nan)

In [949]:
df.isna().sum()

Профиль      0
Вид_п        0
Имя_п        5
Имя_п0       5
Символ      37
Вес          2
Поверхн      2
Тип_п        2
Разрешен     0
дл_имя       5
dtype: int64

In [950]:
df = df[df["Имя_п"].isna() == False]

In [951]:
df = df[df["Вес"].isna() == False]

In [952]:
df["Имя_п"] = df["Имя_п"].str.strip()
df["Имя_п0"] = df["Имя_п0"].str.strip()

In [953]:
df = df.drop(["Имя_п0", "дл_имя"], axis=1)

In [954]:
df = df[df["Разрешен"]]
df = df.drop(["Разрешен"], axis=1)

In [955]:
df = df[(df["Имя_п"] != "200*160*7") | (df["Символ"].isna() == False)]

In [956]:
df = df.drop_duplicates(subset=["Вид_п", "Имя_п", "Символ"])

In [957]:
df.loc[df["Вид_п"] == 16, "Символ"] = "-"
df.loc[df["Вид_п"] == 17, "Символ"] = "-"

In [958]:
df_2 = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Тс_элементы', db_conn)

In [959]:
df_2["Имя_п"] = df_2["Имя_п"].str.strip()
df_2["Символ"] = df_2["Символ"].str.strip()

In [960]:
c_ids = pd.read_csv("construction_ids.csv")
c_ids = c_ids.drop(["Unnamed: 0"], axis=1)
c_ids = c_ids.values.flatten().tolist()

In [961]:
df_2 = df_2[df_2["Сч_констр"].isin(c_ids)]

In [962]:
p_ids = pd.read_csv("prof.csv")
p_ids = p_ids.drop(["Unnamed: 0"], axis=1)
p_ids = p_ids.values.flatten().tolist()

In [963]:
df_2 = df_2[df_2["Вид_п"].isin(p_ids)]

In [964]:
df_2 = df_2[df_2["Символ"].isna() == False]

In [965]:
df_2 = df_2.drop(["МаркаОЗ", "АрифмВыр"], axis=1)

In [966]:
df_2 = df_2.drop(["ПерекрПерим", "КолВСеч", "Поз", "ГруппаПТ"], axis=1)

In [967]:
for index, row in df_2.iterrows():
    try:
        df_2.loc[index, "profile_id"] = df.loc[(df["Вид_п"] == row["Вид_п"]) & (df["Имя_п"] == row["Имя_п"]) & (df["Символ"] == row["Символ"]), "Профиль"].values[0]
    except Exception as e:
        pass

In [968]:
df_null = df_2.loc[df_2['profile_id'].isna()]

In [969]:
df_null = df_null.drop(["Сч_констр", "Сч_поз", "Сталь", "Длина", "статус", "profile_id"], axis=1)

In [970]:
df_null = df_null.rename(columns={"Вид_п": "class_id",
                        "Имя_п": "name",
                        "Символ": "symbol",
                        "Вес": "weight",
                        "Поверхн": "area",
                        "Тип_п": "type_id"
                       })

In [971]:
df_null.insert(0, 'id', range(1962, len(df_null) + 1962))

In [972]:
df = df.where(pd.notnull(df), None)

In [973]:
df = df.rename(columns={"Профиль": "id",
                        "Вид_п": "class_id",
                        "Имя_п": "name",
                        "Символ": "symbol",
                        "Вес": "weight",
                        "Поверхн": "area",
                        "Тип_п": "type_id"
                       })

In [974]:
df = df.append(df_null, ignore_index = True)

In [975]:
df[df.duplicated(subset=["class_id", "name", "symbol"], keep="first")]
df = df.drop_duplicates(subset=["class_id", "name", "symbol"])

In [976]:
df = df[df["id"] != 3194]

In [977]:
df.loc[df["id"] == 2755, "weight"] = 4.396
df.loc[df["id"] == 2755, "area"] = 0.00288

In [978]:
df.loc[:, ["id", "class_id", "name", "symbol"]].to_csv("profiles.csv")

In [979]:
cursor.execute(open("sql_server/33.sql", "r").read())
conn.commit()

In [980]:
execute_values(df, "profiles")

execute_values() done


<h1>Марки стали</h1>

In [982]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Спр_стали', db_conn)

In [983]:
df = df.replace("", np.nan)

In [984]:
df = df.drop([71])

In [985]:
df["Имя_с"] = df["Имя_с"].str.strip()
df["Стандарт"] = df["Стандарт"].str.strip()

In [986]:
df = df.rename(columns={"Шифр_с": "id",
                        "Имя_с": "name",
                        "Стандарт": "standard",
                        "Прочность": "strength"
                       })

In [987]:
df = df.where(pd.notnull(df), None)

In [988]:
cursor.execute(open("sql_server/41.sql", "r").read())
conn.commit()

In [989]:
execute_values(df, "steel")

execute_values() done


<h1>Перечень элементов</h1>

In [991]:
df = pd.read_sql_query('SELECT * FROM Pro_t.dbo.Тс_элементы', db_conn)

In [992]:
df["Имя_п"] = df["Имя_п"].str.strip()
df["Символ"] = df["Символ"].str.strip()

In [993]:
c_ids = pd.read_csv("construction_ids.csv")
c_ids = c_ids.drop(["Unnamed: 0"], axis=1)
c_ids = c_ids.values.flatten().tolist()

In [994]:
df = df[df["Сч_констр"].isin(c_ids)]

In [995]:
p_ids = pd.read_csv("prof.csv")
p_ids = p_ids.drop(["Unnamed: 0"], axis=1)
p_ids = p_ids.values.flatten().tolist()

In [996]:
df = df[df["Вид_п"].isin(p_ids)]

In [997]:
df.loc[df["Сталь"] == 72, "Сталь"] = 36

In [998]:
df = df[df["Символ"].isna() == False]

In [999]:
df = df.drop(["МаркаОЗ", "АрифмВыр"], axis=1)

In [1000]:
df = df.drop(["ПерекрПерим", "КолВСеч", "Поз", "ГруппаПТ"], axis=1)

In [1001]:
profiles = pd.read_csv("profiles.csv")
profiles = profiles.drop(["Unnamed: 0"], axis=1)

In [1002]:
df.loc[df["Сч_поз"] == 179045, "Имя_п"] = "140*4"
df.loc[df["Сч_поз"] == 179034, "Имя_п"] = "140*4"

In [1003]:
for index, row in df.iterrows():
    try:
        df.loc[index, "profile_id"] = profiles.loc[(profiles["class_id"] == row["Вид_п"]) & (profiles["name"] == row["Имя_п"]) & (profiles["symbol"] == row["Символ"]), "id"].values[0]
    except:
        pass

In [1004]:
df = df.drop(["Имя_п", "Символ", "Вес", "Поверхн", "Тип_п", "статус", "Вид_п"], axis=1)

In [1005]:
df = df.rename(columns={"Сч_констр": "construction_id",
                        "Сч_поз": "id",
                        "Сталь": "steel_id",
                        "Длина": "length"
                       })

In [1006]:
cursor.execute(open("sql_server/48.sql", "r").read())
conn.commit()

In [1007]:
execute_values(df, "construction_elements")

execute_values() done


<h1>Разделы общих указаний</h1>

In [1009]:
df = pd.DataFrame({'id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17],
                   'name': ["Разработано на основании",
                            "Разработано в соответствии",
                            "Перечень технических регламентов",
                            "Высотная отметка",
                            "Климатические условия",
                            "Конструктивные решения",
                            "Характеристики конструкций",
                            "Материал конструкций",
                            "Указания по разработке КМД и ППР",
                            "Указания по изготовлению и монтажу",
                            "Крепление элементов",
                            "Соединение элементов",
                            "Антикоррозионная защита",
                            "Огнезащита конструкций",
                            "Перечень скрытых видов работ",
                            "Перечень ответственных конструкций",
                            "Коэффициент трудозатрат"],
                   'order_num': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]})

In [1010]:
cursor.execute(open("sql_server/37_2.sql", "r").read())
conn.commit()

In [1011]:
execute_values(df, "general_data_sections")

execute_values() done


<h1>Пункты общих указаний марки</h1>

In [1013]:
df = pd.DataFrame({
#     'id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73],
                   'mark_id': [4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320],
                   'section_id': [1, 1, 1, 1, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 8, 8, 9, 9, 10, 10, 10, 10, 11, 11, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 13, 13, 13, 13, 15, 15, 15, 15],
                   'text': ["Рабочая документация разработана на основании:",
                            "- поручения от 28.02.2020 № АГЛ-35/0362",
                            "- письма от 05.03.2020 № УПиН-35/0431",
                            "- рекомендаций, выполненного специалистами \"АО МАГНИТОГОРСКИЙ ГИПРОМЕЗ\"",
                            "Рабочая документация разработана в соответствии с техническими условиями, требованиями действующих технических регламентов, стандартов, сводов правил и других документов, содержащих установленные требования.",
                            "Перечень технических регламентов и нормативных документов, в соответствии с требованиями которых разработана рабочая документация",
                            "- Федеральный закон от 30.12.2009 № 384-ФЗ \"Технический регламент о безопасности зданий и сооружений\"",
                            "- Федеральный закон от 22.07.2008 № 123-ФЗ \"Технический регламент о требованиях пожарной безопасности\"",
                            "- СП 16.13330.2017 \"Стальные конструкции. Актуализированная редакция СНиП II-23-81*\"",
                            "- СП 20.13330.2016 \"Нагрузки и воздействия. Актуализированная редакция СНиП 2.01.07-85*\"",
                            "- СП 28.13330.2017 \"Защита строительных конструкций от коррозии. Актуализированная редакция СНиП 2.03.11-85\"",
                            "- СП 53-102-2004 \"Общие правила проектирования стальных конструкций\"",
                            "- СП 70.13330.2012 \"Несущие и ограждающие конструкции. Актуализированная редакция СНиП 3.03.01-87\"",
                            "- ГОСТ 23118-2012 \"Конструкции стальные строительные. Общие технические условия\"",
                            "- ГОСТ 27751-2014 \"Надежность строительных конструкций и оснований. Основные положения\"",
                            "- ГОСТ 9.402-2004 \"Единая система защиты от коррозии и старения (ЕСЗКС). Покрытия лакокрасочные. Подготовка металлических поверхностей к окрашиванию\"",
                            "Высотные отметки абсолютные.",
                            "Климатические условия",
                            "- климатический район – IB",
                            "- расчетная температура наружного воздуха наиболее холодных суток обеспеченностью 0,98 – минус 43 °C",
                            "- нормативное значение веса снегового покрова на 1 м^2 горизонтальной поверхности земли для г. Магнитогорска (согласно СП 20.13330-2016, приложение K, табл. K1) – 1,3 кПа (130 кг/м^2)",
                            "- нормативное значение ветрового давления для II района (согласно СП 20.13330-2016, п. 11.14, табл. 11.1) – 0,30 кПа (30 кг/м^2)",
                            "- зона влажности – сухая",
                            "- расчетная сейсмическая активность – 5 баллов",
                            "Конструктивные решения",
                            "# Чертежи марки \"КМ\" являются исходным материалом для разработки деталировочных чертежей марки \"КМД\"",
                            "# В данной рабочей документации разработано восстановление пролетных строений (усиление ферм пролетных строений, металлоконструкций по верхним и нижним поясам ферм) и восстановление антикоррозионного покрытия существующих элементов пролетных строений галереи конвейера № 706-1 УУК и КДИ аглоцеха ГОП ПАО \"ММК\"",
                            "# Все работы должны выполняться специализированной организацией, имеющей свидетельство СРО на данный вид деятельности и имеющие допуск по электробезопасности",
                            "# Работы производить по специально разработанному и согласованному в установленном порядке со всеми инспектирующими организациями и эксплуатацией цеха проекту производства работ (ППР) с соблюдением требований действующих нормативных документов и обязательным учетом фактического состояния существующих конструкций",
                            "# При выполнении работ принять меры по обеспечению целостности и сохранности существующих сооружений и коммуникаций, находящихся в зоне производства работ",
                            "# При производстве сварочных работ предусмотреть комплекс организационно-технических мероприятий по обеспечению пожарной безопасности",
                            "# На всех этапах ведения демонтажных и монтажных работ обеспечить устойчивость и неизменяемость положения существующих конструкций",
                            "# Перед началом работ необходимо выполнить очистку конструкций в зоне ремонта от мусора, грязи, технологической пыли",
                            "# Исполнитель несет ответственность за соответствие выполненных работ чертежам. Всякое отступление от чертежей при производстве работ должно быть согласовано с организацией-разработчиком данной рабочей документации",
                            "Характеристики конструкций",
                            "- коэффициент надежности по ответственности – {}",
                            "- степень агрессивного воздействия среды – {}",
                            "- расчетная температура эксплуатации – {}",
                            "Материал конструкций",
                            "Марки сталей элементов конструкций приняты в зависимости от группы конструкций с учетом расчетной температуры и приведены в таблицах \"Ведомость элементов\", на схемах расположения элементов конструкций, на чертежах элементов и узлов конструкций и в спецификации металлопроката",
                            "Указания по разработке КМД и ППР",
                            "Отступления от рабочих чертежей КМ не допускаются. В случае необходимости эти отступления должны быть согласованы с организацией-разработчиком рабочих чертежей марки \"КМ\"",
                            "Указания по изготовлению и монтажу металлоконструкций",
                            "# Изготовление металлоконструкций производить в соответствии с требованиями ГОСТ 23118-2012, СП 53-101-98 \"Изготовление и контроль качества стальных строительных конструкций\", указаниями в чертежах марки \"КМ\"",
                            "# Монтаж металлоконструкций производить в соответствии с требованиями СП 70.13330.2012 \"Несущие и ограждающие конструкции. Актуализированная редакция СНиП 3.03.01-87\", СНИП 12-04-2002 \"Безопасность труда в строительстве. Часть 2. Строительное производство\" и указаниями в чертежах марки \"КМ\"",
                            "# Отклонения металлоконструкций при изготовлении и монтаже не должны превышать предельных значений, указанных в СП 53-101-98, СП 70.13330.2012 и в чертежах марки \"КМ\" данного основного комплекта",
                            "Крепление элементов",
                            "Минимальное усилие для расчетного прикрепления элементов – 5 тс",
                            "Соединение элементов металлоконструкций",
                            "- сварные",
                            "- на болтах нормальной точности по ГОСТ Р ИСО 4014-2013",
                            "# Материалы для сварки принимать по таблице Г.1 СП 16.13330.2017 в зависимости от марок стали свариваемых элементов",
                            "# Монтажные швы выполнять ручной сваркой ГОСТ 9467-75 \"Электроды покрытые металлические для ручной дуговой сварки конструкционных и теплоустойчивых сталей. Типы\":\nДля сталей C245 сварку производить электродами типа 342A",
                            "# Применение прерывистых швов не допускается",
                            "# Контроль качества монтажных сварных соединений должен осуществляться в соответствии с требованиями п. 10.4 СП 70.13330.2012",
                            "# Болты класса точности B (нормальной точности) M20 приняты по ГОСТ Р ИСО 4014-2013 \"Болты с шестигранной головкой. Классы точности A и B\", класса прочности 8.8 по ГОСТ ISO 898-1-2014 \"Механические свойства крепежных изделий из углеродистых и легированных сталей. Часть 1. Болты, винты и шпильки установленных классов прочности с крупным и мелким шагом резьбы\"",
                            "# Гайки тип 1 класса точности B приняты по ГОСТ ISO 4032-2014. При работе болтов на срез и растяжение класс прочности гаек принят в соответствии с классом прочности болтов: 5 – при 5.6",
                            "Шайбы приняты круглые по ГОСТ 11371-78 исполнения 1 класса точности A",
                            "Для болтов класса точности B диаметр отверстий в конструкциях, работающих на срез и смятие, принят d=dв+2",
                            "В расчетных соединениях для болтов класса точности B разность номинальных диаметров отверстий и болтов не должна превышать 3 мм с учетом предельных отклонений диаметра отверстия и их овальности 0; +0,6; и 0; +1,0 по табл. 2 ГОСТ 23118-2012",
                            "Антикоррозионная защита",
                            "# Антикоррозионная защита запроектирована в соответствии с требованиями СП 28.13330.2017 \"Защита строительных конструкций от коррозии\"",
                            "# Антикоррозионную защиту производить в соответствии с требованиями СП 72.13330.2016 \"Защита строительных конструкций и сооружений от коррозии. СНиП 3.04.03-85\" и ГОСТ 12.3.005-75 \"Система стандартов безопасности труда (ССБТ). Работы окрасочные. Общие требования безопасности\"",
                            "# Подготовка поверхностей перед окраской должна осуществляться в соответствии с требованиями ГОСТ 9.402-2004 путем удаления ржавчины и прокатной окалины с помощью ручного или механизированного абразивного инструмента и очистки жировых и прочих загрязнений с помощью волосяных кистей или ветоши, смоченных в уайт-спирите. Перед нанесением антикоррозионного покрытия поверхности конструкций обеспылить",
                            "# Защита металлоконструкций от коррозии осуществляется окраской лакокрасочными материалами группы III: эмалью XB-7141 по ТУ 2313-016-75351875-2005 в 4 слоя по 2 слоям грунтовки XC-010 по ТУ 6-21-7-89",
                            "Общая толщина покрытия – 160 мкм",
                            "Степень очистки поверхности стальных конструкций от окислов перед окраской – 2 по ГОСТ 9.402-2004",
                            "Допускается замена лакокрасочных материалов на сертифицированные материалы с аналогичными техническими характеристиками",
                            "Цвет окраски антикоррозионного покрытия принимать в соответствии с цветом окраски существующих конструкций",
                            "Перечень видов работ, на которые должны быть составлены акты освидетельствования скрытых работ:",
                            "- подготовка поверхности металлоконструкций под окраску",
                            "- приемка грунтовочного слоя антикоррозионного покрытия",
                            "- визуальный контроль качества сварных швов"],
                   'order_num': [1, 2, 3, 4, 1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 1, 1, 2, 3, 4, 5, 6, 7, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 1, 2, 1, 2, 1, 2, 3, 4, 1, 2, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 1, 2, 3, 4]})


In [1014]:
df.insert(0, 'id', range(1, len(df) + 1))

In [1015]:
cursor.execute(open("sql_server/35_xx.sql", "r").read())
conn.commit()

In [1016]:
execute_values(df, "mark_general_data_points")

execute_values() done


<h1>Первый запуск до этой клетки, потом нужно программно добавить пользователей</h1>

<h1>Пункты общих указаний пользователя (id seq fail)</h1>

In [2263]:
df = pd.DataFrame({
#     'id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73],
                   'user_id': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                   'section_id': [1, 1, 1, 1, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 8, 8, 9, 9, 10, 10, 10, 10, 11, 11, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 13, 13, 13, 13, 15, 15, 15, 15],
                   'text': ["Рабочая документация разработана на основании:",
                            "- поручения от 28.02.2020 № АГЛ-35/0362",
                            "- письма от 05.03.2020 № УПиН-35/0431",
                            "- рекомендаций, выполненного специалистами \"АО МАГНИТОГОРСКИЙ ГИПРОМЕЗ\"",
                            "Рабочая документация разработана в соответствии с техническими условиями, требованиями действующих технических регламентов, стандартов, сводов правил и других документов, содержащих установленные требования.",
                            "Перечень технических регламентов и нормативных документов, в соответствии с требованиями которых разработана рабочая документация",
                            "- Федеральный закон от 30.12.2009 № 384-ФЗ \"Технический регламент о безопасности зданий и сооружений\"",
                            "- Федеральный закон от 22.07.2008 № 123-ФЗ \"Технический регламент о требованиях пожарной безопасности\"",
                            "- СП 16.13330.2017 \"Стальные конструкции. Актуализированная редакция СНиП II-23-81*\"",
                            "- СП 20.13330.2016 \"Нагрузки и воздействия. Актуализированная редакция СНиП 2.01.07-85*\"",
                            "- СП 28.13330.2017 \"Защита строительных конструкций от коррозии. Актуализированная редакция СНиП 2.03.11-85\"",
                            "- СП 53-102-2004 \"Общие правила проектирования стальных конструкций\"",
                            "- СП 70.13330.2012 \"Несущие и ограждающие конструкции. Актуализированная редакция СНиП 3.03.01-87\"",
                            "- ГОСТ 23118-2012 \"Конструкции стальные строительные. Общие технические условия\"",
                            "- ГОСТ 27751-2014 \"Надежность строительных конструкций и оснований. Основные положения\"",
                            "- ГОСТ 9.402-2004 \"Единая система защиты от коррозии и старения (ЕСЗКС). Покрытия лакокрасочные. Подготовка металлических поверхностей к окрашиванию\"",
                            "Высотные отметки абсолютные.",
                            "Климатические условия",
                            "- климатический район – IB",
                            "- расчетная температура наружного воздуха наиболее холодных суток обеспеченностью 0,98 – минус 43 °C",
                            "- нормативное значение веса снегового покрова на 1 м^2 горизонтальной поверхности земли для г. Магнитогорска (согласно СП 20.13330-2016, приложение K, табл. K1) – 1,3 кПа (130 кг/м^2)",
                            "- нормативное значение ветрового давления для II района (согласно СП 20.13330-2016, п. 11.14, табл. 11.1) – 0,30 кПа (30 кг/м^2)",
                            "- зона влажности – сухая",
                            "- расчетная сейсмическая активность – 5 баллов",
                            "Конструктивные решения",
                            "# Чертежи марки \"КМ\" являются исходным материалом для разработки деталировочных чертежей марки \"КМД\"",
                            "# В данной рабочей документации разработано восстановление пролетных строений (усиление ферм пролетных строений, металлоконструкций по верхним и нижним поясам ферм) и восстановление антикоррозионного покрытия существующих элементов пролетных строений галереи конвейера № 706-1 УУК и КДИ аглоцеха ГОП ПАО \"ММК\"",
                            "# Все работы должны выполняться специализированной организацией, имеющей свидетельство СРО на данный вид деятельности и имеющие допуск по электробезопасности",
                            "# Работы производить по специально разработанному и согласованному в установленном порядке со всеми инспектирующими организациями и эксплуатацией цеха проекту производства работ (ППР) с соблюдением требований действующих нормативных документов и обязательным учетом фактического состояния существующих конструкций",
                            "# При выполнении работ принять меры по обеспечению целостности и сохранности существующих сооружений и коммуникаций, находящихся в зоне производства работ",
                            "# При производстве сварочных работ предусмотреть комплекс организационно-технических мероприятий по обеспечению пожарной безопасности",
                            "# На всех этапах ведения демонтажных и монтажных работ обеспечить устойчивость и неизменяемость положения существующих конструкций",
                            "# Перед началом работ необходимо выполнить очистку конструкций в зоне ремонта от мусора, грязи, технологической пыли",
                            "# Исполнитель несет ответственность за соответствие выполненных работ чертежам. Всякое отступление от чертежей при производстве работ должно быть согласовано с организацией-разработчиком данной рабочей документации",
                            "Характеристики конструкций",
                            "- коэффициент надежности по ответственности – {}",
                            "- степень агрессивного воздействия среды – {}",
                            "- расчетная температура эксплуатации – {}",
                            "Материал конструкций",
                            "Марки сталей элементов конструкций приняты в зависимости от группы конструкций с учетом расчетной температуры и приведены в таблицах \"Ведомость элементов\", на схемах расположения элементов конструкций, на чертежах элементов и узлов конструкций и в спецификации металлопроката",
                            "Указания по разработке КМД и ППР",
                            "Отступления от рабочих чертежей КМ не допускаются. В случае необходимости эти отступления должны быть согласованы с организацией-разработчиком рабочих чертежей марки \"КМ\"",
                            "Указания по изготовлению и монтажу металлоконструкций",
                            "# Изготовление металлоконструкций производить в соответствии с требованиями ГОСТ 23118-2012, СП 53-101-98 \"Изготовление и контроль качества стальных строительных конструкций\", указаниями в чертежах марки \"КМ\"",
                            "# Монтаж металлоконструкций производить в соответствии с требованиями СП 70.13330.2012 \"Несущие и ограждающие конструкции. Актуализированная редакция СНиП 3.03.01-87\", СНИП 12-04-2002 \"Безопасность труда в строительстве. Часть 2. Строительное производство\" и указаниями в чертежах марки \"КМ\"",
                            "# Отклонения металлоконструкций при изготовлении и монтаже не должны превышать предельных значений, указанных в СП 53-101-98, СП 70.13330.2012 и в чертежах марки \"КМ\" данного основного комплекта",
                            "Крепление элементов",
                            "Минимальное усилие для расчетного прикрепления элементов – 5 тс",
                            "Соединение элементов металлоконструкций",
                            "- сварные",
                            "- на болтах нормальной точности по ГОСТ Р ИСО 4014-2013",
                            "# Материалы для сварки принимать по таблице Г.1 СП 16.13330.2017 в зависимости от марок стали свариваемых элементов",
                            "# Монтажные швы выполнять ручной сваркой ГОСТ 9467-75 \"Электроды покрытые металлические для ручной дуговой сварки конструкционных и теплоустойчивых сталей. Типы\":\nДля сталей C245 сварку производить электродами типа 342A",
                            "# Применение прерывистых швов не допускается",
                            "# Контроль качества монтажных сварных соединений должен осуществляться в соответствии с требованиями п. 10.4 СП 70.13330.2012",
                            "# Болты класса точности B (нормальной точности) M20 приняты по ГОСТ Р ИСО 4014-2013 \"Болты с шестигранной головкой. Классы точности A и B\", класса прочности 8.8 по ГОСТ ISO 898-1-2014 \"Механические свойства крепежных изделий из углеродистых и легированных сталей. Часть 1. Болты, винты и шпильки установленных классов прочности с крупным и мелким шагом резьбы\"",
                            "# Гайки тип 1 класса точности B приняты по ГОСТ ISO 4032-2014. При работе болтов на срез и растяжение класс прочности гаек принят в соответствии с классом прочности болтов: 5 – при 5.6",
                            "Шайбы приняты круглые по ГОСТ 11371-78 исполнения 1 класса точности A",
                            "Для болтов класса точности B диаметр отверстий в конструкциях, работающих на срез и смятие, принят d=dв+2",
                            "В расчетных соединениях для болтов класса точности B разность номинальных диаметров отверстий и болтов не должна превышать 3 мм с учетом предельных отклонений диаметра отверстия и их овальности 0; +0,6; и 0; +1,0 по табл. 2 ГОСТ 23118-2012",
                            
                            
                            "Антикоррозионная защита",
                            "# Антикоррозионная защита запроектирована в соответствии с требованиями СП 28.13330.2017 \"Защита строительных конструкций от коррозии\"",
                            "# Антикоррозионную защиту производить в соответствии с требованиями СП 72.13330.2016 \"Защита строительных конструкций и сооружений от коррозии. СНиП 3.04.03-85\" и ГОСТ 12.3.005-75 \"Система стандартов безопасности труда (ССБТ). Работы окрасочные. Общие требования безопасности\"",
                            "# Подготовка поверхностей перед окраской должна осуществляться в соответствии с требованиями ГОСТ 9.402-2004 путем удаления ржавчины и прокатной окалины с помощью ручного или механизированного абразивного инструмента и очистки жировых и прочих загрязнений с помощью волосяных кистей или ветоши, смоченных в уайт-спирите. Перед нанесением антикоррозионного покрытия поверхности конструкций обеспылить",
                            "# Защита металлоконструкций от коррозии осуществляется окраской лакокрасочными материалами группы III: эмалью XB-7141 по ТУ 2313-016-75351875-2005 в 4 слоя по 2 слоям грунтовки XC-010 по ТУ 6-21-7-89",
                            "Общая толщина покрытия – 160 мкм",
                            "Степень очистки поверхности стальных конструкций от окислов перед окраской – 2 по ГОСТ 9.402-2004",
                            "Допускается замена лакокрасочных материалов на сертифицированные материалы с аналогичными техническими характеристиками",
                            "Цвет окраски антикоррозионного покрытия принимать в соответствии с цветом окраски существующих конструкций",
                            "Перечень видов работ, на которые должны быть составлены акты освидетельствования скрытых работ:",
                            "- подготовка поверхности металлоконструкций под окраску",
                            "- приемка грунтовочного слоя антикоррозионного покрытия",
                            "- визуальный контроль качества сварных швов"],
                   'order_num': [1, 2, 3, 4, 1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 1, 1, 2, 3, 4, 5, 6, 7, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 1, 2, 1, 2, 1, 2, 3, 4, 1, 2, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 1, 2, 3, 4]})

In [2264]:
df.insert(0, 'id', range(1, len(df) + 1))

In [2265]:
cursor.execute(open("sql_server/35_2.sql", "r").read())
conn.commit()

Will fail first time. Need to create users.

In [2266]:
execute_values(df, "general_data_points")

execute_values() done


In [626]:
conn.close()